In [1]:
import pandas as pd
import csv
import numpy as np
from collections import Counter
import re
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import pickle

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.cluster import KMeans

In [3]:
clusters = {}

In [4]:
take_sample = True
sample_size = 0.05

In [82]:
def select_k(data):
    kmeanModel = KMeans()
    r = [i for i in range (20, 300, 10)]
    print(r)
    visualizer = KElbowVisualizer(kmeanModel, k=r, timings=False)
    visualizer.fit(data)
    k = visualizer.show()
    
def dump_clusters(kmeans, data, suffix):
    for i in range(kmeans.n_clusters):
        cluster_data = data[np.where(kmeans.labels_ == i)].ravel()
        min_ = np.min(cluster_data)
        max_ = np.max(cluster_data)
        clusters['cluster-{}-{}'.format(i, suffix)] = [kmeans.cluster_centers_[i].squeeze().tolist(), min_, max_]

### People

In [6]:
people = pd.read_csv("data/people.csv")
people.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,first_name,last_name,...,state_code,region,city,featured_job_organization_uuid,featured_job_organization_name,featured_job_title,facebook_url,linkedin_url,twitter_url,logo_url
0,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,person,ben-elowitz,https://www.crunchbase.com/person/ben-elowitz,7876.0,2007-05-25 14:03:54,2021-03-25 03:59:21,Ben,Elowitz,...,WA,Washington,Seattle,cf253887-5eac-21a2-28d3-47db7311f7e9,Madrona Venture Group,Managing Director,http://www.facebook.com/elowitz,https://www.linkedin.com/in/elowitz,http://twitter.com/elowitz,https://res.cloudinary.com/crunchbase-producti...
1,5ceca97b-493c-1446-6249-5aaa33464763,Kevin Flaherty,person,kevin-flaherty,https://www.crunchbase.com/person/kevin-flaherty,154638.0,2007-05-25 14:04:16,2018-02-12 23:14:37,Kevin,Flaherty,...,NaN,NaN,NaN,789e5e4d-0c90-d06e-92a0-b800b461c3da,DRSmedia,Team Member,NaN,http://www.linkedin.com/in/kevinflaherty,https://twitter.com/tallkp,https://res.cloudinary.com/crunchbase-producti...
2,9f99a98a-aa97-b30b-0d36-db67c1d277e0,Raju Vegesna,person,raju-vegesna,https://www.crunchbase.com/person/raju-vegesna,2497.0,2007-05-26 02:33:03,2021-02-16 10:41:17,Raju,Vegesna,...,CA,California,San Francisco,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,Chief Evangelist,NaN,https://www.linkedin.com/in/raju-vegesna-78a6541/,NaN,https://res.cloudinary.com/crunchbase-producti...
3,6e1bca72-a865-b518-b305-31214ce2d1b0,Ian Wenig,person,ian-wenig,https://www.crunchbase.com/person/ian-wenig,65330.0,2007-05-26 02:34:53,2021-03-23 10:25:04,Ian,Wenig,...,NaN,NaN,NaN,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,VP Business Development,NaN,NaN,NaN,https://res.cloudinary.com/crunchbase-producti...
4,3b598c59-7b6c-2d48-763c-da55bca77035,Owen Byrne,person,owen-byrne,https://www.crunchbase.com/person/owen-byrne,131555.0,2007-05-26 03:06:45,2018-02-12 22:14:28,Owen,Byrne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://twitter.com/owenbyrne,https://res.cloudinary.com/crunchbase-producti...


In [7]:
people["idslice"] = people["uuid"].apply(lambda x: x[-5:])
people["name"] = people["name"].apply(lambda x: "_".join(x.split()))
people["name_id"] = people["name"] + "@" + people["idslice"]
print("total number of people:", len(people))
print("total number of unique name_id's:", len(people["name_id"].unique()))
if take_sample:
    people = people.loc[:int(sample_size*len(people))]
people.head()

total number of people: 1271523
total number of unique name_id's: 1271523


,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,first_name,last_name,...,city,featured_job_organization_uuid,featured_job_organization_name,featured_job_title,facebook_url,linkedin_url,twitter_url,logo_url,idslice,name_id
0,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben_Elowitz,person,ben-elowitz,https://www.crunchbase.com/person/ben-elowitz,7876.0,2007-05-25 14:03:54,2021-03-25 03:59:21,Ben,Elowitz,...,Seattle,cf253887-5eac-21a2-28d3-47db7311f7e9,Madrona Venture Group,Managing Director,http://www.facebook.com/elowitz,https://www.linkedin.com/in/elowitz,http://twitter.com/elowitz,https://res.cloudinary.com/crunchbase-producti...,37a71,Ben_Elowitz@37a71
1,5ceca97b-493c-1446-6249-5aaa33464763,Kevin_Flaherty,person,kevin-flaherty,https://www.crunchbase.com/person/kevin-flaherty,154638.0,2007-05-25 14:04:16,2018-02-12 23:14:37,Kevin,Flaherty,...,NaN,789e5e4d-0c90-d06e-92a0-b800b461c3da,DRSmedia,Team Member,NaN,http://www.linkedin.com/in/kevinflaherty,https://twitter.com/tallkp,https://res.cloudinary.com/crunchbase-producti...,64763,Kevin_Flaherty@64763
2,9f99a98a-aa97-b30b-0d36-db67c1d277e0,Raju_Vegesna,person,raju-vegesna,https://www.crunchbase.com/person/raju-vegesna,2497.0,2007-05-26 02:33:03,2021-02-16 10:41:17,Raju,Vegesna,...,San Francisco,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,Chief Evangelist,NaN,https://www.linkedin.com/in/raju-vegesna-78a6541/,NaN,https://res.cloudinary.com/crunchbase-producti...,277e0,Raju_Vegesna@277e0
3,6e1bca72-a865-b518-b305-31214ce2d1b0,Ian_Wenig,person,ian-wenig,https://www.crunchbase.com/person/ian-wenig,65330.0,2007-05-26 02:34:53,2021-03-23 10:25:04,Ian,Wenig,...,NaN,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,VP Business Development,NaN,NaN,NaN,https://res.cloudinary.com/crunchbase-producti...,2d1b0,Ian_Wenig@2d1b0
4,3b598c59-7b6c-2d48-763c-da55bca77035,Owen_Byrne,person,owen-byrne,https://www.crunchbase.com/person/owen-byrne,131555.0,2007-05-26 03:06:45,2018-02-12 22:14:28,Owen,Byrne,...,NaN,NaN,NaN,NaN,NaN,NaN,http://twitter.com/owenbyrne,https://res.cloudinary.com/crunchbase-producti...,77035,Owen_Byrne@77035


In [8]:
#person_type = pd.DataFrame(people[["name_id", "type"]])
#person_type.insert(1, "relation", "type_of")
#person_type

In [9]:
#person_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [10]:
person_gender = pd.DataFrame(people[["name_id", "gender"]])
print("all genders:", person_gender.gender.unique())
# remove NaN and "not_provided"
person_gender["gender"] = person_gender["gender"].replace("not_provided", float("NaN"))
person_gender = person_gender.dropna()
person_gender.insert(1, "relation", "gender")
person_gender

all genders: ['male' 'female' 'not_provided' nan 'non_binary' 'androgynous']


,name_id,relation,gender
0,Ben_Elowitz@37a71,gender,male
1,Kevin_Flaherty@64763,gender,male
2,Raju_Vegesna@277e0,gender,male
3,Ian_Wenig@2d1b0,gender,male
4,Owen_Byrne@77035,gender,male
...,...,...,...
63572,Greg_Martin@c460a,gender,male
63573,Al_Angrisani@be5aa,gender,male
63574,Robert_Douglass@af35a,gender,male
63575,Klaas_Van_Waesberghe@c53bd,gender,male


In [11]:
person_gender.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
person_gender.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [12]:
#gender = pd.DataFrame(person_gender.gender.unique()).drop_duplicates()
#gender.insert(1, "relation", "type_of")
#gender.insert(2, "type", "gnder")
#gender

In [13]:
#gender.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [14]:
person_country = pd.DataFrame(people[["name_id", "country_code"]])
person_country = person_country.dropna()
person_country.insert(1, "relation", "country_code")
person_country

,name_id,relation,country_code
0,Ben_Elowitz@37a71,country_code,USA
2,Raju_Vegesna@277e0,country_code,USA
6,Mark_Zuckerberg@03f22,country_code,USA
7,Dustin_Moskovitz@2cd64,country_code,USA
8,Owen_Van_Natta@94e53,country_code,USA
...,...,...,...
63558,Corey_Capasso@a011e,country_code,USA
63560,Dasha_Zhukova@f074f,country_code,USA
63561,Wendi_Murdoch@e7dc2,country_code,USA
63566,Tom_Camps@9995e,country_code,CAN


In [15]:
person_country.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
person_country.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [16]:
person_job = pd.DataFrame(people[["name_id", "featured_job_title"]])
person_job = person_job.dropna()

In [17]:
person_job["featured_job_title"] = person_job["featured_job_title"].replace("co founder", "co-founder")
person_job["featured_job_title"] = person_job["featured_job_title"].replace(",", " ")
person_job["featured_job_title"] = person_job["featured_job_title"].apply(lambda x: ' ' + x.lower() + ' ')

In [18]:
jobs = []
with open('job_title_dictionary.txt') as f:
    lines = f.readlines()
    for line in lines:
        jobs.append(line.split('\t')[0].lower())

In [19]:
print(len(jobs))
len(person_job)

36423


48726

In [20]:
def match_job(x):
    matches = []
    #print(x)
    for j in jobs:
        if ' ' + j + ' ' in x:
            matches.append(j)
    return matches

In [21]:
############ Run to create jobs file
person_job["featured_job_title"] = person_job["featured_job_title"].apply(lambda x: match_job(x))
person_job = person_job.dropna()
person_job = person_job.explode("featured_job_title")
person_job.to_csv("jobs.txt", sep='\t', mode='a', index=False, header = None)

In [22]:
with open("jobs.txt", encoding="mbcs") as f:
    lines = f.readlines()
    jobs = [l.strip().split('\t') for l in lines]
    jobs = [j for j in jobs if len(j) == 2]
len(jobs)

108612

In [23]:
person_job = pd.DataFrame(jobs)
person_job.columns = ["name_id", "featured_job_title"]
person_job.insert(1, "relation", "featured_job")

In [24]:
person_job

,name_id,relation,featured_job_title
0,Ben_Elowitz@37a71,featured_job,director
1,Ben_Elowitz@37a71,featured_job,managing
2,Ben_Elowitz@37a71,featured_job,managing director
3,Kevin_Flaherty@64763,featured_job,member
4,Kevin_Flaherty@64763,featured_job,team
...,...,...,...
108607,Robert_Douglass@af35a,featured_job,founder
108608,Robert_Douglass@af35a,featured_job,founder
108609,Klaas_Van_Waesberghe@c53bd,featured_job,founder
108610,Klaas_Van_Waesberghe@c53bd,featured_job,founder


In [25]:
person_job.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
person_job.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [26]:
jj = Counter(person_job["featured_job_title"])

In [27]:
#job = pd.DataFrame(list(jj.keys())).drop_duplicates()
#job.insert(1, "relation", "type_of")
#job.insert(2, "type", "job")
#job

In [28]:
#job.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [29]:
# check if the featured job correspond to different organizations, false means they correspond to a unique org
person_org = pd.DataFrame(people[["name_id", "featured_job_organization_uuid"]])
person_org = person_org.dropna()
person_org["check"] = person_org["featured_job_organization_uuid"].apply(lambda x: True if "," in x else False)
print(person_org.check.unique())

[False]


In [30]:
people2 = pd.DataFrame(people[["name_id", "featured_job_organization_uuid", "featured_job_organization_name"]])
people2 = people2.dropna()
people2["org_idslice"] = people2["featured_job_organization_uuid"].apply(lambda x: x[-5:])
people2["org_name_id"] = people2["featured_job_organization_name"] + "@" + people2["org_idslice"]
people2["org_name_id"] = people2["org_name_id"].apply(lambda x: "_".join(x.split()))
person_org = pd.DataFrame(people2[["name_id", "org_name_id"]])
person_org = person_org.dropna()
person_org.insert(1, "relation", "featured_job_organization")
person_org

,name_id,relation,org_name_id
0,Ben_Elowitz@37a71,featured_job_organization,Madrona_Venture_Group@1f7e9
1,Kevin_Flaherty@64763,featured_job_organization,DRSmedia@1c3da
2,Raju_Vegesna@277e0,featured_job_organization,Zoho@7efc7
3,Ian_Wenig@2d1b0,featured_job_organization,Zoho@7efc7
5,Ron_Gorodetzky@10ea5,featured_job_organization,Fflick@c173b
...,...,...,...
63570,Jon_Cook@94311,featured_job_organization,HowRandom@4cd04
63573,Al_Angrisani@be5aa,featured_job_organization,Thumb@c14cd
63574,Robert_Douglass@af35a,featured_job_organization,Open_Goldberg_Project@b6e06
63575,Klaas_Van_Waesberghe@c53bd,featured_job_organization,KartMe@e88dc


In [31]:
person_org.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
person_org.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

### Organization

In [32]:
org = pd.read_csv("data/organizations.csv")
org.columns

Index(['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'legal_name', 'roles', 'domain', 'homepage_url',
       'country_code', 'state_code', 'region', 'city', 'address',
       'postal_code', 'status', 'short_description', 'category_list',
       'category_groups_list', 'num_funding_rounds', 'total_funding_usd',
       'total_funding', 'total_funding_currency_code', 'founded_on',
       'last_funding_on', 'closed_on', 'employee_count', 'email', 'phone',
       'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'alias1',
       'alias2', 'alias3', 'primary_role', 'num_exits'],
      dtype='object')

In [33]:
org = org[org["name"].notna()]
org["idslice"] = org["uuid"].apply(lambda x: x[-5:])
org["name_id"] = org["name"] + "@" + org["idslice"]
org["name_id"] = org["name_id"].apply(lambda x: "_".join(x.split()))
print("name_id unique? ", len(org["name_id"].unique()) == len(org))
if take_sample:
    org = org.loc[:int(sample_size*len(org))]
org.head()

name_id unique?  True


,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,...,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits,idslice,name_id
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,143488.0,2007-05-25 13:51:27,2021-07-22 04:02:55,NaN,company,...,NaN,https://twitter.com/wetpainttv,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN,33abd,Wetpaint@33abd
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,5448.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",...,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,1.0,7efc7,Zoho@7efc7
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,15055.0,2007-05-26 03:03:23,2018-12-10 10:09:14,"Digg Holdings, LLC",company,...,http://www.linkedin.com/company/digg,http://twitter.com/digg,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN,553d0,Digg@553d0
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,144529.0,2007-05-26 03:21:34,2019-06-19 12:17:48,NaN,investor,...,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,investor,45.0,a8eec,Omidyar_Network@a8eec
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,11.0,2007-05-26 04:22:15,2021-10-11 11:59:53,"Facebook, Inc.","investor,company",...,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://res.cloudinary.com/crunchbase-producti...,NaN,NaN,NaN,company,NaN,04fbb,Facebook@04fbb


In [34]:
org_em = pd.DataFrame(org[["name_id", "employee_count"]])
org_em.insert(1, "relation", "employee_count")
org_em

,name_id,relation,employee_count
0,Wetpaint@33abd,employee_count,51-100
1,Zoho@7efc7,employee_count,1001-5000
2,Digg@553d0,employee_count,51-100
3,Omidyar_Network@a8eec,employee_count,101-250
4,Facebook@04fbb,employee_count,10000+
...,...,...,...
82103,LLamasoft@eeb8d,employee_count,501-1000
82104,Zokos@1be7a,employee_count,1-10
82105,Evry@0bf31,employee_count,1-10
82106,Laurizon_Experiences@f7cd3,employee_count,1-10


In [35]:
org_em.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_em.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [36]:
#org_type = pd.DataFrame(org[["name_id", "type"]])
#org_type.dropna()
#org_type.insert(1, "relation", "type_of")
#print(org_type)

In [37]:
#org_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [38]:
org_role = pd.DataFrame(org[["name_id", "roles"]])
org_role = org_role.dropna()
org_role["roles"] = org_role["roles"].apply(lambda x: x.split(","))
org_role = org_role.explode("roles")
org_role["roles"] = org_role["roles"].apply(lambda x: x.strip())
org_role.insert(1, "relation", "role")
print(org_role["roles"].unique())
print(org_role)

['company' 'investor' 'school']
                          name_id relation     roles
0                  Wetpaint@33abd     role   company
1                      Zoho@7efc7     role  investor
1                      Zoho@7efc7     role   company
2                      Digg@553d0     role   company
3           Omidyar_Network@a8eec     role  investor
...                           ...      ...       ...
82103             LLamasoft@eeb8d     role   company
82104                 Zokos@1be7a     role   company
82105                  Evry@0bf31     role   company
82106  Laurizon_Experiences@f7cd3     role   company
82107                Adorii@78793     role   company

[85719 rows x 3 columns]


In [39]:
org_role.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_role.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [40]:
org_prole = pd.DataFrame(org[["name_id", "primary_role"]])
org_prole = org_prole.dropna()
print(org_prole.primary_role.unique())
org_prole.insert(1, "relation", "primary_role")
print(org_prole)

['company' 'investor' 'school']
                          name_id      relation primary_role
0                  Wetpaint@33abd  primary_role      company
1                      Zoho@7efc7  primary_role      company
2                      Digg@553d0  primary_role      company
3           Omidyar_Network@a8eec  primary_role     investor
4                  Facebook@04fbb  primary_role      company
...                           ...           ...          ...
82103             LLamasoft@eeb8d  primary_role      company
82104                 Zokos@1be7a  primary_role      company
82105                  Evry@0bf31  primary_role      company
82106  Laurizon_Experiences@f7cd3  primary_role      company
82107                Adorii@78793  primary_role      company

[82106 rows x 3 columns]


In [41]:
org_role.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_role.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [42]:
#roles = org_prole.primary_role.unique()
#roles = pd.DataFrame(roles).drop_duplicates()
#roles.insert(1, "relation", "type_of")
#roles.insert(2, "type", "role")
#roles

In [43]:
#roles.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [44]:
org_coun = pd.DataFrame(org[["name_id", "country_code"]])
org_coun = org_coun.dropna()
org_coun.insert(1, "relation", "country_code")
print(org_coun)

                          name_id      relation country_code
0                  Wetpaint@33abd  country_code          USA
1                      Zoho@7efc7  country_code          USA
2                      Digg@553d0  country_code          USA
3           Omidyar_Network@a8eec  country_code          USA
4                  Facebook@04fbb  country_code          USA
...                           ...           ...          ...
82103             LLamasoft@eeb8d  country_code          USA
82104                 Zokos@1be7a  country_code          USA
82105                  Evry@0bf31  country_code          USA
82106  Laurizon_Experiences@f7cd3  country_code          CAN
82107                Adorii@78793  country_code          USA

[74394 rows x 3 columns]


In [45]:
org_coun.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_coun.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [46]:
org_status = pd.DataFrame(org[["name_id", "status"]])
org_status = org_status.dropna()
org_status['status'] = org_status['status'].apply(lambda x: 'status@' + x)
status = org_status.status.unique()
org_status.insert(1, "relation", "org_status")
print(org_status)

                          name_id    relation            status
0                  Wetpaint@33abd  org_status   status@acquired
1                      Zoho@7efc7  org_status  status@operating
2                      Digg@553d0  org_status   status@acquired
3           Omidyar_Network@a8eec  org_status  status@operating
4                  Facebook@04fbb  org_status        status@ipo
...                           ...         ...               ...
82103             LLamasoft@eeb8d  org_status     status@closed
82104                 Zokos@1be7a  org_status  status@operating
82105                  Evry@0bf31  org_status  status@operating
82106  Laurizon_Experiences@f7cd3  org_status  status@operating
82107                Adorii@78793  org_status  status@operating

[82106 rows x 3 columns]


In [47]:
org_status.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_status.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [48]:
#status = pd.DataFrame(status).drop_duplicates()
#status.insert(1, "relation", "type_of")
#status.insert(2, "type", "status")
#status

In [49]:
#status.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [50]:
org_c = pd.DataFrame(org[["name_id", "category_list"]])
org_c = org_c.dropna()
org_c["category_list"] = org_c["category_list"].apply(lambda x: x.split(","))
org_c = org_c.explode("category_list")
org_c["category_list"] = org_c["category_list"].replace("Other", float("NaN"))
org_c = org_c.dropna()
org_c["category_list"] = org_c["category_list"].apply(lambda x: "_".join(x.split()))
org_c.insert(1, "relation", "category")
print(len(org_c["category_list"].unique()))
print(org_c)

716
                          name_id  relation            category_list
0                  Wetpaint@33abd  category               Publishing
0                  Wetpaint@33abd  category             Social_Media
0                  Wetpaint@33abd  category  Social_Media_Management
1                      Zoho@7efc7  category          Cloud_Computing
1                      Zoho@7efc7  category            Collaboration
...                           ...       ...                      ...
82105                  Evry@0bf31  category                 Payments
82105                  Evry@0bf31  category                   Social
82106  Laurizon_Experiences@f7cd3  category                      SEO
82107                Adorii@78793  category               E-Commerce
82107                Adorii@78793  category                  Fashion

[249339 rows x 3 columns]


In [51]:
org_c.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_c.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [52]:
category = pd.read_csv("data/category_groups.csv")
category.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,category_groups_list
0,f9b14a15-5517-8f38-0562-729ebb54dfdb,Homeless Shelter,category,homeless-shelter,https://www.crunchbase.com/category/homeless-s...,NaN,2014-04-20 07:58:46,2018-02-13 00:59:11,Other
1,f8320fcf-b657-37d2-1495-daa3ad888ece,Freemium,category,freemium,https://www.crunchbase.com/category/freemium,NaN,2014-04-20 07:58:46,2018-02-13 00:59:11,Other
2,f0193199-a968-b457-eb49-95344e22a5ce,Industrial,category,industrial,https://www.crunchbase.com/category/industrial,NaN,2014-04-20 07:58:46,2018-02-13 00:59:11,Manufacturing
3,ed8217ff-bd11-26fe-d82e-eb98a276dc2e,Innovation Management,category,innovation-management,https://www.crunchbase.com/category/innovation...,NaN,2014-04-20 07:58:46,2018-02-13 00:59:11,Professional Services
4,e9794581-9547-2150-8185-7b747f5c9913,Lighting,category,lighting,https://www.crunchbase.com/category/lighting,NaN,2014-04-20 07:58:46,2018-02-13 00:59:11,Hardware


In [53]:
c_sub = category[["name", "category_groups_list"]]
c_sub["name"] = c_sub["name"].apply(lambda x: "_".join(x.split()))
c_sub["category_groups_list"] = c_sub["category_groups_list"].apply(lambda x: x.split(","))
c_sub = c_sub.explode("category_groups_list")
c_sub["category_groups_list"] = c_sub["category_groups_list"].replace("Other", float("NaN"))
c_sub = c_sub.dropna()
c_sub["category_groups_list"] = c_sub["category_groups_list"].apply(lambda x: "_".join(x.split()))
c_sub.insert(1, "relation", "category_group")
c_sub

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,name,relation,category_groups_list
2,Industrial,category_group,Manufacturing
3,Innovation_Management,category_group,Professional_Services
4,Lighting,category_group,Hardware
5,Mobile,category_group,Mobile
6,Tutoring,category_group,Education
...,...,...,...
739,Quantum_Computing,category_group,Science_and_Engineering
740,Marine_Technology,category_group,Science_and_Engineering
741,Smart_Cities,category_group,Real_Estate
742,Last_Mile_Transportation,category_group,Transportation


In [54]:
c_sub.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
c_sub.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [55]:
#categories = org_c["category_list"].tolist()
#categories.extend(c_sub["name"])
#category_type = pd.DataFrame(categories)
#category_type = category_type.drop_duplicates(keep = "first")
#category_type.insert(1, "relation", "type_of")
#category_type.insert(2, "type", "category")
#category_type

In [56]:
#category_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [57]:
#categoryg_type = pd.DataFrame(c_sub["category_groups_list"].unique())
#categoryg_type.insert(1, "relation", "type_of")
#categoryg_type.insert(2, "type", "category_group")
#categoryg_type

In [58]:
#category_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [59]:
#subtype = pd.DataFrame([['category', 'subtype_of', 'category_group']])
#subtype.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [60]:
org_fd = pd.DataFrame(org[["name_id", "founded_on"]])
org_fd = org_fd.dropna()
org_fd['founded_on'] = org_fd['founded_on'].apply(lambda x: x[:4])
org_fd.insert(1, "relation", "founded_on_year")
org_fd

,name_id,relation,founded_on
0,Wetpaint@33abd,founded_on_year,2005
1,Zoho@7efc7,founded_on_year,1996
2,Digg@553d0,founded_on_year,2004
3,Omidyar_Network@a8eec,founded_on_year,2004
4,Facebook@04fbb,founded_on_year,2004
...,...,...,...
82103,LLamasoft@eeb8d,founded_on_year,2002
82104,Zokos@1be7a,founded_on_year,2011
82105,Evry@0bf31,founded_on_year,2012
82106,Laurizon_Experiences@f7cd3,founded_on_year,2012


In [61]:
org_fd.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_fd.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

### Ipos

In [62]:
ipos = pd.read_csv("data/ipos.csv")
ipos.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,org_uuid,org_name,...,went_public_on,share_price_usd,share_price,share_price_currency_code,valuation_price_usd,valuation_price,valuation_price_currency_code,money_raised_usd,money_raised,money_raised_currency_code
0,72d30ebd-53ef-2486-6c29-22785c5173ce,NaN,ipo,microsoft-ipo--72d30ebd,https://www.crunchbase.com/ipo/microsoft-ipo--...,10555.0,2008-02-09 05:25:18,2018-02-12 23:11:05,fd80725f-53fc-7009-9878-aeecf1e9ffbb,Microsoft,...,1986-03-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,NaN,ipo,the-walt-disney-company-ipo--3ad2b068,https://www.crunchbase.com/ipo/the-walt-disney...,16334.0,2008-02-09 05:40:32,2019-02-25 22:31:49,756936c0-c335-f0ae-0a3d-fe26bdff5695,The Walt Disney Company,...,1978-01-13,NaN,NaN,NaN,NaN,NaN,NaN,300000000.0,300000000.0,USD
2,a265c6f6-4b96-4079-096a-967a37f3da2b,NaN,ipo,divx-ipo--a265c6f6,https://www.crunchbase.com/ipo/divx-ipo--a265c6f6,41263.0,2008-02-25 23:52:11,2018-02-12 23:57:54,73296f0d-85a5-78d5-90b3-86c5f8981ba9,DivX,...,2006-10-22,16.0,16.0,USD,1.600000e+08,1.600000e+08,USD,145000000.0,145000000.0,USD
3,ee426509-826e-5dd0-9309-e79c8f384904,NaN,ipo,xo-group-ipo--ee426509,https://www.crunchbase.com/ipo/xo-group-ipo--e...,43883.0,2008-02-29 00:31:34,2018-02-12 23:41:42,ff8439cf-097c-a88a-9bb9-dd83d23aa14b,XO Group,...,1999-12-02,NaN,NaN,NaN,NaN,NaN,NaN,35000000.0,35000000.0,USD
4,66b57372-77f0-ec5f-ca33-0175fff79b0e,NaN,ipo,bmc-software-ipo--66b57372,https://www.crunchbase.com/ipo/bmc-software-ip...,36176.0,2008-03-18 14:07:10,2020-12-09 11:02:40,ab8e5ba4-df5d-121b-93b6-eae7a0c89245,BMC Software,...,1988-08-12,NaN,NaN,NaN,6.000000e+09,6.000000e+09,USD,NaN,NaN,NaN


In [63]:
ipos["idslice"] = ipos["org_uuid"].apply(lambda x: x[-5:])
ipos["name_id"] = ipos["org_name"] + "@" + ipos["idslice"]
ipos["uuid"] = ipos["uuid"].apply(lambda x: "ipo@" + x)  # mark it as ipo
ipos["name_id"] = ipos["name_id"].apply(lambda x: "_".join(x.split()))
print(ipos.columns)
if take_sample:
    ipos = ipos.loc[:int(sample_size*len(ipos))]
ipos.head()

Index(['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'org_uuid', 'org_name', 'org_cb_url', 'country_code',
       'state_code', 'region', 'city', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code', 'idslice', 'name_id'],
      dtype='object')


,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,org_uuid,org_name,...,share_price,share_price_currency_code,valuation_price_usd,valuation_price,valuation_price_currency_code,money_raised_usd,money_raised,money_raised_currency_code,idslice,name_id
0,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce,NaN,ipo,microsoft-ipo--72d30ebd,https://www.crunchbase.com/ipo/microsoft-ipo--...,10555.0,2008-02-09 05:25:18,2018-02-12 23:11:05,fd80725f-53fc-7009-9878-aeecf1e9ffbb,Microsoft,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9ffbb,Microsoft@9ffbb
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,NaN,ipo,the-walt-disney-company-ipo--3ad2b068,https://www.crunchbase.com/ipo/the-walt-disney...,16334.0,2008-02-09 05:40:32,2019-02-25 22:31:49,756936c0-c335-f0ae-0a3d-fe26bdff5695,The Walt Disney Company,...,NaN,NaN,NaN,NaN,NaN,300000000.0,300000000.0,USD,f5695,The_Walt_Disney_Company@f5695
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,NaN,ipo,divx-ipo--a265c6f6,https://www.crunchbase.com/ipo/divx-ipo--a265c6f6,41263.0,2008-02-25 23:52:11,2018-02-12 23:57:54,73296f0d-85a5-78d5-90b3-86c5f8981ba9,DivX,...,16.0,USD,1.600000e+08,1.600000e+08,USD,145000000.0,145000000.0,USD,81ba9,DivX@81ba9
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,NaN,ipo,xo-group-ipo--ee426509,https://www.crunchbase.com/ipo/xo-group-ipo--e...,43883.0,2008-02-29 00:31:34,2018-02-12 23:41:42,ff8439cf-097c-a88a-9bb9-dd83d23aa14b,XO Group,...,NaN,NaN,NaN,NaN,NaN,35000000.0,35000000.0,USD,aa14b,XO_Group@aa14b
4,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e,NaN,ipo,bmc-software-ipo--66b57372,https://www.crunchbase.com/ipo/bmc-software-ip...,36176.0,2008-03-18 14:07:10,2020-12-09 11:02:40,ab8e5ba4-df5d-121b-93b6-eae7a0c89245,BMC Software,...,NaN,NaN,6.000000e+09,6.000000e+09,USD,NaN,NaN,NaN,89245,BMC_Software@89245


In [64]:
# check uniqueness
print(len(ipos))
print(len(ipos.name_id.unique()))

# a company can have multple ipos and it is not unique, so we use uuid of ipos instead
#ipo_type = pd.DataFrame(ipos[["uuid", "type"]])
#ipo_type.insert(1, "relation", "type_of")
#print(ipo_type)

2356
2356


In [65]:
#ipo_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [66]:
org_ipo = pd.DataFrame(ipos[["name_id", "uuid"]])
org_ipo = org_ipo.dropna()
org_ipo.insert(1, "relation", "org_ipo")
org_ipo

,name_id,relation,uuid
0,Microsoft@9ffbb,org_ipo,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce
1,The_Walt_Disney_Company@f5695,org_ipo,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4
2,DivX@81ba9,org_ipo,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b
3,XO_Group@aa14b,org_ipo,ipo@ee426509-826e-5dd0-9309-e79c8f384904
4,BMC_Software@89245,org_ipo,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e
...,...,...,...
2351,Open_Solutions@bd702,org_ipo,ipo@ca0f86df-2db3-648b-5931-1684753c74ca
2352,Crystal_Rock_Holdings@12335,org_ipo,ipo@30698140-9ba2-0434-138d-c96f5ff5c421
2353,OpenTable@b5484,org_ipo,ipo@60123005-f02d-bb72-4b6c-431ad781a657
2354,Orbitz_Worldwide@4b959,org_ipo,ipo@261b1dd9-5ca8-2185-b76a-d573bc7a6a0b


In [67]:
org_ipo.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
org_ipo.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [68]:
ipo_coun = pd.DataFrame(ipos[["uuid", "country_code"]])
ipo_coun = ipo_coun.dropna()
ipo_coun.insert(1, "relation", "country_code")
ipo_coun

,uuid,relation,country_code
0,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce,country_code,USA
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,country_code,USA
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,country_code,USA
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,country_code,USA
4,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e,country_code,USA
...,...,...,...
2351,ipo@ca0f86df-2db3-648b-5931-1684753c74ca,country_code,USA
2352,ipo@30698140-9ba2-0434-138d-c96f5ff5c421,country_code,USA
2353,ipo@60123005-f02d-bb72-4b6c-431ad781a657,country_code,USA
2354,ipo@261b1dd9-5ca8-2185-b76a-d573bc7a6a0b,country_code,USA


In [69]:
ipo_coun.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
ipo_coun.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [70]:
ipo_st = pd.DataFrame(ipos[["uuid", "stock_exchange_symbol"]])
ipo_st = ipo_st.dropna()
ipo_st.insert(1, "relation", "stock_exchange_symbol")
print(ipo_st.stock_exchange_symbol.unique())
ipo_st["stock_exchange_symbol"] = ipo_st["stock_exchange_symbol"].apply(lambda x: "stock@" + x)
ipo_st = ipo_st.dropna()
ipo_st

['nasdaq' 'nyse' 'otcpink' 'tsx' 'etr' 'hkg' 'lse' 'nysemkt' 'bit' 'tyo'
 'otcqb' 'otcbb' 'ams' 'ose' 'tlv' 'hel' 'otcqx' 'jse' 'kosdaq' 'cve'
 'nse' 'fwb' 'ist' 'nysearca' 'eli' 'ebr' 'asx' 'sto' 'tpe' 'swx' 'sse'
 'epa' 'jp' 'szse' 'moex' 'krx' 'bmv' 'bme' 'bom' 'amex']


,uuid,relation,stock_exchange_symbol
0,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce,stock_exchange_symbol,stock@nasdaq
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,stock_exchange_symbol,stock@nyse
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,stock_exchange_symbol,stock@nasdaq
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,stock_exchange_symbol,stock@nyse
4,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e,stock_exchange_symbol,stock@nasdaq
...,...,...,...
2351,ipo@ca0f86df-2db3-648b-5931-1684753c74ca,stock_exchange_symbol,stock@nasdaq
2352,ipo@30698140-9ba2-0434-138d-c96f5ff5c421,stock_exchange_symbol,stock@nysemkt
2353,ipo@60123005-f02d-bb72-4b6c-431ad781a657,stock_exchange_symbol,stock@nasdaq
2354,ipo@261b1dd9-5ca8-2185-b76a-d573bc7a6a0b,stock_exchange_symbol,stock@nyse


In [71]:
ipo_st.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
ipo_st.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [72]:
#symbol = ipo_st["stock_exchange_symbol"].unique()
#s_type = pd.DataFrame(symbol)
#s_type.insert(1, "relation", "type_of")
#s_type.insert(2, "type", "stock_exchange_symbol")
#s_type

In [73]:
#s_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [74]:
ipo_wp = pd.DataFrame(ipos[["uuid", "went_public_on"]])
ipo_wp = ipo_wp.dropna()
ipo_wpm = ipo_wp.copy() 
ipo_wp["went_public_on"] = ipo_wp["went_public_on"].apply(lambda x: x[:4])
ipo_wp.insert(1, "relation", "went_public_on_year")
ipo_wp

,uuid,relation,went_public_on
0,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce,went_public_on_year,1986
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,went_public_on_year,1978
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,went_public_on_year,2006
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,went_public_on_year,1999
4,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e,went_public_on_year,1988
...,...,...,...
2351,ipo@ca0f86df-2db3-648b-5931-1684753c74ca,went_public_on_year,2003
2352,ipo@30698140-9ba2-0434-138d-c96f5ff5c421,went_public_on_year,2003
2353,ipo@60123005-f02d-bb72-4b6c-431ad781a657,went_public_on_year,2009
2354,ipo@261b1dd9-5ca8-2185-b76a-d573bc7a6a0b,went_public_on_year,2007


In [75]:
ipo_wp.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
ipo_wp.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [76]:
ipo_wpm["went_public_on"] = ipo_wpm["went_public_on"].apply(lambda x: int(x[5:7]))
ipo_wpm.insert(1, "relation", "went_public_on_month")
ipo_wpm

,uuid,relation,went_public_on
0,ipo@72d30ebd-53ef-2486-6c29-22785c5173ce,went_public_on_month,3
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,went_public_on_month,1
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,went_public_on_month,10
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,went_public_on_month,12
4,ipo@66b57372-77f0-ec5f-ca33-0175fff79b0e,went_public_on_month,8
...,...,...,...
2351,ipo@ca0f86df-2db3-648b-5931-1684753c74ca,went_public_on_month,11
2352,ipo@30698140-9ba2-0434-138d-c96f5ff5c421,went_public_on_month,1
2353,ipo@60123005-f02d-bb72-4b6c-431ad781a657,went_public_on_month,5
2354,ipo@261b1dd9-5ca8-2185-b76a-d573bc7a6a0b,went_public_on_month,7


In [77]:
ipo_wpm.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
ipo_wpm.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [78]:
ipo_sp = pd.DataFrame(ipos[["uuid", "share_price_usd"]])
ipo_sp = ipo_sp.dropna()
ipo_sp.insert(1, "relation", "share_price_usd")
print(ipo_sp)

                                          uuid         relation  \
2     ipo@a265c6f6-4b96-4079-096a-967a37f3da2b  share_price_usd   
11    ipo@7fddf98a-43d4-0ab2-8ed6-e57e5141049c  share_price_usd   
12    ipo@7394891a-4d02-f62e-9543-b2cf02b6455c  share_price_usd   
21    ipo@53e384d5-e529-0b19-25a8-e8a7f3c2a6a3  share_price_usd   
23    ipo@8306e349-3540-bbe2-305d-1ce2ee72a1f4  share_price_usd   
...                                        ...              ...   
2347  ipo@0f37f882-975f-e801-5cbf-5f1439f95bd5  share_price_usd   
2350  ipo@71ddb61a-6720-fbcd-34ce-fba767d24b22  share_price_usd   
2351  ipo@ca0f86df-2db3-648b-5931-1684753c74ca  share_price_usd   
2353  ipo@60123005-f02d-bb72-4b6c-431ad781a657  share_price_usd   
2355  ipo@62128d6b-1652-af0a-f0fb-51c1502340f1  share_price_usd   

      share_price_usd  
2               16.00  
11              27.00  
12              13.00  
21              14.00  
23              26.25  
...               ...  
2347            18.00  
235

In [79]:
ipo_sp.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [84]:
column = 'share_price_usd'
data = np.array(ipo_sp[column].tolist()).reshape((len(ipo_sp), 1))
select_k(data) #comment of sample

In [85]:
final_kmeans = KMeans(n_clusters=60).fit(data) # for sample set 10
labels = ['cluster-{}-{}'.format(l, column) for l in final_kmeans.labels_]
mapping = dict(zip(data.squeeze().tolist(), labels))
ipo_sp[column] = ipo_sp[column].map(mapping)
ipo_sp.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
dump_clusters(final_kmeans, data, column)
ipo_sp

,uuid,relation,share_price_usd
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,share_price_usd,cluster-7-share_price_usd
11,ipo@7fddf98a-43d4-0ab2-8ed6-e57e5141049c,share_price_usd,cluster-8-share_price_usd
12,ipo@7394891a-4d02-f62e-9543-b2cf02b6455c,share_price_usd,cluster-3-share_price_usd
21,ipo@53e384d5-e529-0b19-25a8-e8a7f3c2a6a3,share_price_usd,cluster-3-share_price_usd
23,ipo@8306e349-3540-bbe2-305d-1ce2ee72a1f4,share_price_usd,cluster-8-share_price_usd
...,...,...,...
2347,ipo@0f37f882-975f-e801-5cbf-5f1439f95bd5,share_price_usd,cluster-7-share_price_usd
2350,ipo@71ddb61a-6720-fbcd-34ce-fba767d24b22,share_price_usd,cluster-7-share_price_usd
2351,ipo@ca0f86df-2db3-648b-5931-1684753c74ca,share_price_usd,cluster-7-share_price_usd
2353,ipo@60123005-f02d-bb72-4b6c-431ad781a657,share_price_usd,cluster-0-share_price_usd


In [86]:
ipo_mr = pd.DataFrame(ipos[["uuid", "money_raised_usd"]])
ipo_mr = ipo_mr.dropna()
ipo_mr.insert(1, "relation", "raised_amount_usd")
ipo_mr

,uuid,relation,money_raised_usd
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,raised_amount_usd,300000000.0
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,raised_amount_usd,145000000.0
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,raised_amount_usd,35000000.0
6,ipo@d1be8921-512c-1077-3cb3-303d0819ba0a,raised_amount_usd,15000000.0
11,ipo@7fddf98a-43d4-0ab2-8ed6-e57e5141049c,raised_amount_usd,109090854.0
...,...,...,...
2350,ipo@71ddb61a-6720-fbcd-34ce-fba767d24b22,raised_amount_usd,84000000.0
2351,ipo@ca0f86df-2db3-648b-5931-1684753c74ca,raised_amount_usd,85000000.0
2353,ipo@60123005-f02d-bb72-4b6c-431ad781a657,raised_amount_usd,60000000.0
2354,ipo@261b1dd9-5ca8-2185-b76a-d573bc7a6a0b,raised_amount_usd,510000000.0


In [87]:
ipo_mr.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [88]:
column = 'money_raised_usd'
data = np.array(ipo_mr[column].tolist()).reshape((len(ipo_mr), 1))
#select_k(data)

In [89]:
final_kmeans = KMeans(n_clusters=60).fit(data)
labels = ['cluster-{}-{}'.format(l, column) for l in final_kmeans.labels_]
mapping = dict(zip(data.squeeze().tolist(), labels))
ipo_mr[column] = ipo_mr[column].map(mapping)
ipo_mr.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
dump_clusters(final_kmeans, data, column)
ipo_mr

,uuid,relation,money_raised_usd
1,ipo@3ad2b068-2d97-f646-0b80-1e5f3d7adfc4,raised_amount_usd,cluster-8-money_raised_usd
2,ipo@a265c6f6-4b96-4079-096a-967a37f3da2b,raised_amount_usd,cluster-6-money_raised_usd
3,ipo@ee426509-826e-5dd0-9309-e79c8f384904,raised_amount_usd,cluster-0-money_raised_usd
6,ipo@d1be8921-512c-1077-3cb3-303d0819ba0a,raised_amount_usd,cluster-0-money_raised_usd
11,ipo@7fddf98a-43d4-0ab2-8ed6-e57e5141049c,raised_amount_usd,cluster-6-money_raised_usd
...,...,...,...
2350,ipo@71ddb61a-6720-fbcd-34ce-fba767d24b22,raised_amount_usd,cluster-0-money_raised_usd
2351,ipo@ca0f86df-2db3-648b-5931-1684753c74ca,raised_amount_usd,cluster-0-money_raised_usd
2353,ipo@60123005-f02d-bb72-4b6c-431ad781a657,raised_amount_usd,cluster-0-money_raised_usd
2354,ipo@261b1dd9-5ca8-2185-b76a-d573bc7a6a0b,raised_amount_usd,cluster-4-money_raised_usd


### Organization parents

In [90]:
org_pa = pd.read_csv("data/org_parents.csv")
org_pa.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,parent_uuid,parent_name
0,d70777cc-14bd-2416-0692-5a483781b78b,Fox Interactive Media,organization,fox-interactive-media,https://www.crunchbase.com/organization/fox-in...,126469.0,2007-06-01 04:46:57,2021-10-08 08:03:37,ddfe6b91-4541-7f5c-7d6a-724f75c98088,21st Century Fox
1,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,Twitter,organization,twitter,https://www.crunchbase.com/organization/twitter,416.0,2007-06-01 15:42:34,2021-01-05 14:26:13,d46e3ee9-8973-d2e9-7f3c-1d5ab5768fe8,Twitter Ventures
2,5858c937-702e-c278-1725-2b5e02471482,Cheung Kong Holdings,organization,cheung-kong-holdings,https://www.crunchbase.com/organization/cheung...,333621.0,2007-06-09 14:20:30,2019-06-24 21:43:04,7ea4b08b-6643-f9a8-20ad-cd3e93d3df26,CK Hutchison Holdings
3,f09c1228-2e7d-1889-6647-ba5021b2e4ea,CBS Entertainment,organization,cbs,https://www.crunchbase.com/organization/cbs,131768.0,2007-06-09 14:30:10,2021-07-31 02:48:06,cd19ea0d-745c-e3b6-d819-30b91caf8ebb,CBS Corporation
4,91f5ca18-e026-9482-8e4f-22593943e0f5,Plaxo,organization,plaxo,https://www.crunchbase.com/organization/plaxo,114198.0,2007-06-09 21:18:54,2020-02-03 09:07:25,fbc7746d-86e0-8fd6-19d8-69280c686d02,Comcast


In [91]:
if take_sample:
    org_pa = org_pa.loc[:int(sample_size*len(org_pa))]

In [92]:
org_pa["idslice"] = org_pa["uuid"].apply(lambda x: x[-5:])
org_pa["name_id"] = org_pa["name"] + "@" +  org_pa["idslice"] 
org_pa["name_id"] = org_pa["name_id"].apply(lambda x: "_".join(x.split()))
org_pa["pidslice"] = org_pa["parent_uuid"].apply(lambda x: x[-5:])
org_pa["pname_id"] = org_pa["parent_name"] +"@" + org_pa["pidslice"]
org_pa["pname_id"] = org_pa["pname_id"].apply(lambda x: "_".join(x.split()))
org_pa.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,parent_uuid,parent_name,idslice,name_id,pidslice,pname_id
0,d70777cc-14bd-2416-0692-5a483781b78b,Fox Interactive Media,organization,fox-interactive-media,https://www.crunchbase.com/organization/fox-in...,126469.0,2007-06-01 04:46:57,2021-10-08 08:03:37,ddfe6b91-4541-7f5c-7d6a-724f75c98088,21st Century Fox,1b78b,Fox_Interactive_Media@1b78b,98088,21st_Century_Fox@98088
1,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,Twitter,organization,twitter,https://www.crunchbase.com/organization/twitter,416.0,2007-06-01 15:42:34,2021-01-05 14:26:13,d46e3ee9-8973-d2e9-7f3c-1d5ab5768fe8,Twitter Ventures,1f709,Twitter@1f709,68fe8,Twitter_Ventures@68fe8
2,5858c937-702e-c278-1725-2b5e02471482,Cheung Kong Holdings,organization,cheung-kong-holdings,https://www.crunchbase.com/organization/cheung...,333621.0,2007-06-09 14:20:30,2019-06-24 21:43:04,7ea4b08b-6643-f9a8-20ad-cd3e93d3df26,CK Hutchison Holdings,71482,Cheung_Kong_Holdings@71482,3df26,CK_Hutchison_Holdings@3df26
3,f09c1228-2e7d-1889-6647-ba5021b2e4ea,CBS Entertainment,organization,cbs,https://www.crunchbase.com/organization/cbs,131768.0,2007-06-09 14:30:10,2021-07-31 02:48:06,cd19ea0d-745c-e3b6-d819-30b91caf8ebb,CBS Corporation,2e4ea,CBS_Entertainment@2e4ea,f8ebb,CBS_Corporation@f8ebb
4,91f5ca18-e026-9482-8e4f-22593943e0f5,Plaxo,organization,plaxo,https://www.crunchbase.com/organization/plaxo,114198.0,2007-06-09 21:18:54,2020-02-03 09:07:25,fbc7746d-86e0-8fd6-19d8-69280c686d02,Comcast,3e0f5,Plaxo@3e0f5,86d02,Comcast@86d02


In [93]:
parent_org = pd.DataFrame(org_pa[["name_id", "pname_id"]])
parent_org = parent_org.dropna()
parent_org.insert(1, "relation", "parent_org")
parent_org

,name_id,relation,pname_id
0,Fox_Interactive_Media@1b78b,parent_org,21st_Century_Fox@98088
1,Twitter@1f709,parent_org,Twitter_Ventures@68fe8
2,Cheung_Kong_Holdings@71482,parent_org,CK_Hutchison_Holdings@3df26
3,CBS_Entertainment@2e4ea,parent_org,CBS_Corporation@f8ebb
4,Plaxo@3e0f5,parent_org,Comcast@86d02
...,...,...,...
976,The_East_Campus_Biotech_Business_Incubator@d387e,parent_org,University_at_Albany@64ebd
977,Traklabs@2d301,parent_org,AKTA@feed4
978,Couri_Orange_Hatchery@08a49,parent_org,Syracuse_University@7bfc9
979,EB_Games@ad7d0,parent_org,GameStop@30406


In [94]:
parent_org.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
parent_org.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

### Funding rounds

In [95]:
fr = pd.read_csv("data/funding_rounds.csv")
fr["uuid"] = fr["uuid"].apply(lambda x: "fr@" + x)
fr["org_idslice"] = fr["org_uuid"].apply(lambda x: x[-5:])
fr["org_name_id"] = fr["org_name"] + "@" + fr["org_idslice"]
fr.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,...,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_uuid,org_name,lead_investor_uuids,org_idslice,org_name_id
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Facebook,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,132354.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,...,USD,NaN,NaN,NaN,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,3f47be49-2e32-8118-01a0-31685a4d0fd7,04fbb,Facebook@04fbb
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,199624.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,...,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36,04fbb,Facebook@04fbb
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,funding_round,facebook-series-b--6fae3958,https://www.crunchbase.com/funding_round/faceb...,190559.0,2007-05-27 06:09:36,2018-02-12 23:30:46,USA,CA,...,USD,502500000.0,502500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-...",04fbb,Facebook@04fbb
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,funding_round,photobucket-series-b--bcd5a63d,https://www.crunchbase.com/funding_round/photo...,269931.0,2007-05-29 11:05:59,2018-02-12 23:27:36,USA,CO,...,USD,NaN,NaN,NaN,2.0,f53cb4de-236e-0b1b-dee8-7104a8b018f9,Photobucket,NaN,018f9,Photobucket@018f9
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,Series A - Geni,funding_round,geni-series-a--60e6afd9,https://www.crunchbase.com/funding_round/geni-...,375185.0,2007-05-31 20:19:28,2018-02-12 23:41:29,USA,CA,...,NaN,10000000.0,10000000.0,USD,1.0,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,fb2f8884-ec07-895a-48d7-d9a9d4d7175c,b3098,Geni@b3098


In [96]:
print(fr.columns)

Index(['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'country_code', 'state_code', 'region', 'city',
       'investment_type', 'announced_on', 'raised_amount_usd', 'raised_amount',
       'raised_amount_currency_code', 'post_money_valuation_usd',
       'post_money_valuation', 'post_money_valuation_currency_code',
       'investor_count', 'org_uuid', 'org_name', 'lead_investor_uuids',
       'org_idslice', 'org_name_id'],
      dtype='object')


In [97]:
if take_sample:
    fr = fr.loc[:int(sample_size*len(fr))]

In [98]:
fr_type = fr[["uuid", "investment_type"]]
fr_type = fr_type.dropna()
fr_type.insert(1, "relation", "investment_type")
fr_type

,uuid,relation,investment_type
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,investment_type,angel
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,investment_type,series_a
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,investment_type,series_b
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,investment_type,series_b
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,investment_type,series_a
...,...,...,...
21897,fr@ba5712fb-36b1-f8a1-1293-f521b658580c,investment_type,seed
21898,fr@76b582a6-3862-0660-f809-1a29422e971e,investment_type,series_unknown
21899,fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99,investment_type,series_unknown
21900,fr@6de059a2-6608-2713-8d21-a4baeb50b9cb,investment_type,series_unknown


In [99]:
fr_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
fr_type.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [100]:
#fr_subtype = pd.DataFrame(fr_type["investment_type"].unique())
#fr_subtype.insert(1, "relation", "subtype_of")
#fr_subtype.insert(2, "type", "funding_round")
#fr_subtype

In [101]:
#fr_subtype.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [102]:
# dictionaries for fetching org names and person names for investor uuids
orgd = dict(zip(org.uuid, org.name_id))
perd = dict(zip(people.uuid, people.name_id))

In [103]:
fr_li = pd.DataFrame(fr[["uuid", "lead_investor_uuids"]])
fr_li = fr_li.dropna()
fr_li["lead_investor_uuids"] = fr_li["lead_investor_uuids"].apply(lambda x: x.split(","))
fr_li = fr_li.explode("lead_investor_uuids")
fr_li["lead_investor_uuids"] = fr_li["lead_investor_uuids"].apply(lambda x: x.strip())
fr_li

,uuid,lead_investor_uuids
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,b08efc27-da40-505a-6f9d-c9e14247bf36
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,e2006571-6b7a-e477-002a-f7014f48a7e3
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,8d5c7e48-82da-3025-dd46-346a31bab86f
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,fb2f8884-ec07-895a-48d7-d9a9d4d7175c
...,...,...
21886,fr@80afe8ea-4fdd-380e-c329-d82b3417d1e4,8c1207b5-dce7-dd47-7e91-e5198b6e939d
21889,fr@612ce5c7-c7ed-7e2f-7a21-815c1a89c958,48150cf5-8e3b-b3aa-1bfb-d1c904fe90df
21897,fr@ba5712fb-36b1-f8a1-1293-f521b658580c,8f2b464f-37c8-f1c3-e3aa-c2b849f84162
21898,fr@76b582a6-3862-0660-f809-1a29422e971e,2f0a3e73-adab-0d47-eff5-cc3bb3007474


In [104]:
def fetch(x, d1, d2):
    if x in d1.keys():
        return d1[x]
    elif x in d2.keys():
        return d2[x]
    else:
        return float("NaN")

In [105]:
fr_li["lead_investors"] = fr_li["lead_investor_uuids"].apply(lambda x: fetch(x, orgd, perd))
fr_li = fr_li.dropna()
fr_li = fr_li[["uuid", "lead_investors"]]
fr_li.insert(1, "relation", "lead_investor")
fr_li = fr_li.dropna()
print(fr_li)

                                          uuid       relation  \
0      fr@8a945939-18e0-cc9d-27b9-bf33817b2818  lead_investor   
1      fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb  lead_investor   
2      fr@6fae3958-a001-27c0-fb7e-666266aedd78  lead_investor   
7      fr@0c000adf-4d17-3ebe-bbfc-74d7e691f9e8  lead_investor   
10     fr@18c9eabc-61c8-fae3-d411-89ac65972882  lead_investor   
...                                        ...            ...   
21880  fr@84c20e8b-33f1-a09d-aedb-5c9f5983e5fc  lead_investor   
21886  fr@80afe8ea-4fdd-380e-c329-d82b3417d1e4  lead_investor   
21897  fr@ba5712fb-36b1-f8a1-1293-f521b658580c  lead_investor   
21898  fr@76b582a6-3862-0660-f809-1a29422e971e  lead_investor   
21899  fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99  lead_investor   

                                    lead_investors  
0                                Peter_Thiel@d0fd7  
1                                      Accel@7bf36  
2                  Meritech_Capital_Partners@ab86f  
7      

In [106]:
fr_li.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
fr_li.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [107]:
fr_coun = pd.DataFrame(fr[["uuid", "country_code"]])
fr_coun = fr_coun.dropna()
fr_coun.insert(1, "relation", "country_code")
fr_coun

,uuid,relation,country_code
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,country_code,USA
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,country_code,USA
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,country_code,USA
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,country_code,USA
4,fr@60e6afd9-1215-465a-dd17-0ed600d4e29b,country_code,USA
...,...,...,...
21897,fr@ba5712fb-36b1-f8a1-1293-f521b658580c,country_code,POL
21898,fr@76b582a6-3862-0660-f809-1a29422e971e,country_code,ESP
21899,fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99,country_code,GBR
21900,fr@6de059a2-6608-2713-8d21-a4baeb50b9cb,country_code,SWE


In [108]:
fr_coun.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
fr_coun.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [109]:
fr_city = fr[["uuid", "city"]]
fr_city = fr_city.dropna()
fr_city["city"] = fr_city["city"].apply(lambda x: "_".join(x.split()))
fr_city.insert(1, "relation", "city")
print(fr_city)

                                          uuid relation            city
0      fr@8a945939-18e0-cc9d-27b9-bf33817b2818     city      Menlo_Park
1      fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb     city      Menlo_Park
2      fr@6fae3958-a001-27c0-fb7e-666266aedd78     city      Menlo_Park
3      fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846     city          Denver
4      fr@60e6afd9-1215-465a-dd17-0ed600d4e29b     city  West_Hollywood
...                                        ...      ...             ...
21897  fr@ba5712fb-36b1-f8a1-1293-f521b658580c     city          Warsaw
21898  fr@76b582a6-3862-0660-f809-1a29422e971e     city       Barcelona
21899  fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99     city         Cannock
21900  fr@6de059a2-6608-2713-8d21-a4baeb50b9cb     city       Stockholm
21901  fr@32b8d975-76a9-3e55-5fe0-f604b30c558d     city         Boulder

[21829 rows x 3 columns]


In [110]:
fr_city.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
fr_city.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [111]:
fr_ao = pd.DataFrame(fr[["uuid", "announced_on"]])
fr_ao = fr_ao.dropna()
fr_aom = fr_ao.copy()
fr_ao["announced_on"] = fr_ao["announced_on"].apply(lambda x: x[:4])
fr_ao.insert(1, "relation", "announced_on_year")
print(fr_ao)

                                          uuid           relation announced_on
0      fr@8a945939-18e0-cc9d-27b9-bf33817b2818  announced_on_year         2004
1      fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb  announced_on_year         2005
2      fr@6fae3958-a001-27c0-fb7e-666266aedd78  announced_on_year         2006
3      fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846  announced_on_year         2006
4      fr@60e6afd9-1215-465a-dd17-0ed600d4e29b  announced_on_year         2007
...                                        ...                ...          ...
21897  fr@ba5712fb-36b1-f8a1-1293-f521b658580c  announced_on_year         2012
21898  fr@76b582a6-3862-0660-f809-1a29422e971e  announced_on_year         2011
21899  fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99  announced_on_year         2011
21900  fr@6de059a2-6608-2713-8d21-a4baeb50b9cb  announced_on_year         2011
21901  fr@32b8d975-76a9-3e55-5fe0-f604b30c558d  announced_on_year         2011

[21902 rows x 3 columns]


In [112]:
fr_ao.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
fr_ao.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [113]:
fr_aom["announced_on"] = fr_aom["announced_on"].apply(lambda x: int(x[5:7]))
fr_aom.insert(1, "relation", "announced_on_month")
print(fr_aom)

                                          uuid            relation  \
0      fr@8a945939-18e0-cc9d-27b9-bf33817b2818  announced_on_month   
1      fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb  announced_on_month   
2      fr@6fae3958-a001-27c0-fb7e-666266aedd78  announced_on_month   
3      fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846  announced_on_month   
4      fr@60e6afd9-1215-465a-dd17-0ed600d4e29b  announced_on_month   
...                                        ...                 ...   
21897  fr@ba5712fb-36b1-f8a1-1293-f521b658580c  announced_on_month   
21898  fr@76b582a6-3862-0660-f809-1a29422e971e  announced_on_month   
21899  fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99  announced_on_month   
21900  fr@6de059a2-6608-2713-8d21-a4baeb50b9cb  announced_on_month   
21901  fr@32b8d975-76a9-3e55-5fe0-f604b30c558d  announced_on_month   

       announced_on  
0                 9  
1                 5  
2                 4  
3                 5  
4                 1  
...             ...  
21897

In [114]:
fr_aom.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
fr_aom.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [115]:
fr_ra = pd.DataFrame(fr[["uuid", "raised_amount_usd"]])
fr_ra = fr_ra.dropna()
fr_ra.insert(1, "relation", "raised_amount_usd")
fr_ra

,uuid,relation,raised_amount_usd
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,raised_amount_usd,500000.0
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,raised_amount_usd,12700000.0
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,raised_amount_usd,27500000.0
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,raised_amount_usd,10500000.0
5,fr@4a56483e-d637-65fa-a2b4-abe6613e5c6b,raised_amount_usd,10000000.0
...,...,...,...
21896,fr@e94e64e8-8fec-ee37-c3ef-9a0d8b791ed5,raised_amount_usd,5000.0
21898,fr@76b582a6-3862-0660-f809-1a29422e971e,raised_amount_usd,4033489.0
21899,fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99,raised_amount_usd,397357.0
21900,fr@6de059a2-6608-2713-8d21-a4baeb50b9cb,raised_amount_usd,1019528.0


In [116]:
fr_ra.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [118]:
column = 'raised_amount_usd'
data = np.array(fr_ra[column].tolist()).reshape((len(fr_ra), 1))
#select_k(data)

In [119]:
final_kmeans = KMeans(n_clusters=60).fit(data)
labels = ['cluster-{}-{}'.format(l, column) for l in final_kmeans.labels_]
mapping = dict(zip(data.squeeze().tolist(), labels))
fr_ra[column] = fr_ra[column].map(mapping)
fr_ra.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
dump_clusters(final_kmeans, data, column)
fr_ra

,uuid,relation,raised_amount_usd
0,fr@8a945939-18e0-cc9d-27b9-bf33817b2818,raised_amount_usd,cluster-0-raised_amount_usd
1,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,raised_amount_usd,cluster-9-raised_amount_usd
2,fr@6fae3958-a001-27c0-fb7e-666266aedd78,raised_amount_usd,cluster-5-raised_amount_usd
3,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,raised_amount_usd,cluster-9-raised_amount_usd
5,fr@4a56483e-d637-65fa-a2b4-abe6613e5c6b,raised_amount_usd,cluster-9-raised_amount_usd
...,...,...,...
21896,fr@e94e64e8-8fec-ee37-c3ef-9a0d8b791ed5,raised_amount_usd,cluster-0-raised_amount_usd
21898,fr@76b582a6-3862-0660-f809-1a29422e971e,raised_amount_usd,cluster-0-raised_amount_usd
21899,fr@e4b0f516-63d3-adb5-f7c6-34930fe6bc99,raised_amount_usd,cluster-0-raised_amount_usd
21900,fr@6de059a2-6608-2713-8d21-a4baeb50b9cb,raised_amount_usd,cluster-0-raised_amount_usd


### Investment partners

In [120]:
ip = pd.read_csv("data/investment_partners.csv")
ip.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,funding_round_uuid,funding_round_name,investor_uuid,investor_name,partner_uuid,partner_name
0,524986f0-3049-54a4-fa72-f60897a5e61d,Accel investment in Series A - Facebook,investment,accel-invested-in-facebook-series-a--d950d7a5-...,https://www.crunchbase.com/investment/accel-in...,NaN,2007-05-27 06:09:10,2018-02-13 00:52:57,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,2d78d1e7-203c-3eb6-bf1b-c51f10e0679b,Jim Breyer
1,524986f0-3049-54a4-fa72-f60897a5e61d,Accel investment in Series A - Facebook,investment,accel-invested-in-facebook-series-a--d950d7a5-...,https://www.crunchbase.com/investment/accel-in...,NaN,2007-05-27 06:09:10,2018-02-13 00:52:57,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,eaf6c243-d355-32f3-e23a-2a5fc82e8b34,Kevin Efrusy
2,6556ab92-6465-25aa-1ffc-7f8b4b09a476,Greylock investment in Series B - Facebook,investment,greylock-invested-in-facebook-series-b--6fae39...,https://www.crunchbase.com/investment/greylock...,NaN,2007-05-27 06:09:36,2018-02-13 00:53:20,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,e2006571-6b7a-e477-002a-f7014f48a7e3,Greylock,478e7efd-bec4-b9f5-304b-cffedc1fc012,David Sze
3,0216e06a-61f8-9cf1-19ba-20811229c53e,Meritech Capital Partners investment in Series...,investment,meritech-capital-partners-invested-in-facebook...,https://www.crunchbase.com/investment/meritech...,NaN,2007-05-27 06:09:36,2019-03-27 19:50:32,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,8d5c7e48-82da-3025-dd46-346a31bab86f,Meritech Capital Partners,0f9f3c05-cb79-f58f-6cc5-98ddc8382d4f,Paul Madera
4,dadd7d86-520d-5e35-3033-fc1d8792ab91,Trinity Ventures investment in Series B - Phot...,investment,trinity-ventures-invested-in-photobucket-serie...,https://www.crunchbase.com/investment/trinity-...,NaN,2007-05-29 11:05:59,2018-02-13 00:55:58,bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,7ca12f7a-2f8e-48b4-a8d1-1a33a0e275b9,Trinity Ventures,ea9f4980-600c-84f4-a5d6-b4f8c2f787fb,Gus Tai


In [121]:
ip["iidslice"] = ip["investor_uuid"].apply(lambda x: x[-5:])
ip["iname_id"] = ip["investor_name"] + "@" + ip["iidslice"]
ip["iname_id"] = ip["iname_id"].apply(lambda x: "_".join(x.split()))
ip["pidslice"] = ip["partner_uuid"].apply(lambda x: x[-5:])
ip["pname_id"] = ip["partner_name"] + "@" + ip["pidslice"]
ip["pname_id"] = ip["pname_id"].apply(lambda x: "_".join(x.split()))
ip["funding_round_uuid"] = ip["funding_round_uuid"].apply(lambda x: "fr" + "@" +  x)
ip.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,funding_round_uuid,funding_round_name,investor_uuid,investor_name,partner_uuid,partner_name,iidslice,iname_id,pidslice,pname_id
0,524986f0-3049-54a4-fa72-f60897a5e61d,Accel investment in Series A - Facebook,investment,accel-invested-in-facebook-series-a--d950d7a5-...,https://www.crunchbase.com/investment/accel-in...,NaN,2007-05-27 06:09:10,2018-02-13 00:52:57,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,2d78d1e7-203c-3eb6-bf1b-c51f10e0679b,Jim Breyer,7bf36,Accel@7bf36,0679b,Jim_Breyer@0679b
1,524986f0-3049-54a4-fa72-f60897a5e61d,Accel investment in Series A - Facebook,investment,accel-invested-in-facebook-series-a--d950d7a5-...,https://www.crunchbase.com/investment/accel-in...,NaN,2007-05-27 06:09:10,2018-02-13 00:52:57,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Facebook,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,eaf6c243-d355-32f3-e23a-2a5fc82e8b34,Kevin Efrusy,7bf36,Accel@7bf36,e8b34,Kevin_Efrusy@e8b34
2,6556ab92-6465-25aa-1ffc-7f8b4b09a476,Greylock investment in Series B - Facebook,investment,greylock-invested-in-facebook-series-b--6fae39...,https://www.crunchbase.com/investment/greylock...,NaN,2007-05-27 06:09:36,2018-02-13 00:53:20,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,e2006571-6b7a-e477-002a-f7014f48a7e3,Greylock,478e7efd-bec4-b9f5-304b-cffedc1fc012,David Sze,8a7e3,Greylock@8a7e3,fc012,David_Sze@fc012
3,0216e06a-61f8-9cf1-19ba-20811229c53e,Meritech Capital Partners investment in Series...,investment,meritech-capital-partners-invested-in-facebook...,https://www.crunchbase.com/investment/meritech...,NaN,2007-05-27 06:09:36,2019-03-27 19:50:32,fr@6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Facebook,8d5c7e48-82da-3025-dd46-346a31bab86f,Meritech Capital Partners,0f9f3c05-cb79-f58f-6cc5-98ddc8382d4f,Paul Madera,ab86f,Meritech_Capital_Partners@ab86f,82d4f,Paul_Madera@82d4f
4,dadd7d86-520d-5e35-3033-fc1d8792ab91,Trinity Ventures investment in Series B - Phot...,investment,trinity-ventures-invested-in-photobucket-serie...,https://www.crunchbase.com/investment/trinity-...,NaN,2007-05-29 11:05:59,2018-02-13 00:55:58,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846,Series B - Photobucket,7ca12f7a-2f8e-48b4-a8d1-1a33a0e275b9,Trinity Ventures,ea9f4980-600c-84f4-a5d6-b4f8c2f787fb,Gus Tai,275b9,Trinity_Ventures@275b9,787fb,Gus_Tai@787fb


In [123]:
if take_sample:
    ip = ip.loc[:int(sample_size*len(ip))]

In [125]:
# check uniqueness of investor
test = ip["investor_uuid"].apply(lambda x: "," in x)
for i in range(len(test)):
    if test[i] == True:
        print(i)
inv = pd.DataFrame(ip["investor_uuid"])
print(len(inv))
inv["investor_uuid"] = inv["investor_uuid"].apply(lambda x: x.split(","))
inv = inv.explode("investor_uuid")
inv["investor_uuid"] = inv["investor_uuid"].apply(lambda x: x.strip())
print(len(inv))

6512
6512


In [126]:
ip_inv = pd.DataFrame(ip[["iname_id", "funding_round_uuid"]])
ip_inv.insert(1, "relation", "funding_round_investor")
print(ip_inv)

                             iname_id                relation  \
0                         Accel@7bf36  funding_round_investor   
1                         Accel@7bf36  funding_round_investor   
2                      Greylock@8a7e3  funding_round_investor   
3     Meritech_Capital_Partners@ab86f  funding_round_investor   
4              Trinity_Ventures@275b9  funding_round_investor   
...                               ...                     ...   
6507         The_Vertical_Group@7f902  funding_round_investor   
6508  New_Leaf_Venture_Partners@484c9  funding_round_investor   
6509        SV_Health_Investors@e1c1c  funding_round_investor   
6510                    Venrock@ea992  funding_round_investor   
6511      U.S._Venture_Partners@7d726  funding_round_investor   

                           funding_round_uuid  
0     fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb  
1     fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb  
2     fr@6fae3958-a001-27c0-fb7e-666266aedd78  
3     fr@6fae3958-a001-27c0

In [127]:
ip_inv.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
ip_inv.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [128]:
ip_parinves = pd.DataFrame(ip[["pname_id", "funding_round_uuid"]])
ip_parinves.insert(1, "relation", "funding_round_investment_partner")
ip_parinves

,pname_id,relation,funding_round_uuid
0,Jim_Breyer@0679b,funding_round_investment_partner,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb
1,Kevin_Efrusy@e8b34,funding_round_investment_partner,fr@d950d7a5-79ff-fb93-ca87-13386b0e2feb
2,David_Sze@fc012,funding_round_investment_partner,fr@6fae3958-a001-27c0-fb7e-666266aedd78
3,Paul_Madera@82d4f,funding_round_investment_partner,fr@6fae3958-a001-27c0-fb7e-666266aedd78
4,Gus_Tai@787fb,funding_round_investment_partner,fr@bcd5a63d-ed99-6963-0dd2-e36f6582f846
...,...,...,...
6507,Jack_W._Lasersohn@79dac,funding_round_investment_partner,fr@55859cd9-665d-bdda-8eba-d643eb70b694
6508,Jeani_Delagardelle@082eb,funding_round_investment_partner,fr@c508b5c5-e0b8-f05f-4b84-334947907bfc
6509,David_Milne@43860,funding_round_investment_partner,fr@c508b5c5-e0b8-f05f-4b84-334947907bfc
6510,David_Pakman@b6cb5,funding_round_investment_partner,fr@d8bddc88-6ee7-f322-e3c9-70249e8d37ad


In [129]:
ip_parinves.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
ip_parinves.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

#### Investors

In [130]:
iv = pd.read_csv("data/investors.csv")
iv.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,roles,domain,...,investment_count,total_funding_usd,total_funding,total_funding_currency_code,founded_on,closed_on,facebook_url,linkedin_url,twitter_url,logo_url
0,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,person,ben-elowitz,https://www.crunchbase.com/person/ben-elowitz,221966.0,2007-05-25 14:03:54,2021-03-25 03:59:21,investor,wetpaint-inc.com,...,1.0,NaN,NaN,NaN,NaN,NaN,http://www.facebook.com/elowitz,https://www.linkedin.com/in/elowitz,http://twitter.com/elowitz,https://res.cloudinary.com/crunchbase-producti...
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,5453.0,2007-05-26 02:30:28,2018-10-27 00:29:49,"investor,company",zoho.com,...,6.0,NaN,NaN,NaN,1996-09-15,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://res.cloudinary.com/crunchbase-producti...
2,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,146604.0,2007-05-26 03:21:34,2019-06-19 12:17:48,investor,omidyar.com,...,327.0,NaN,NaN,NaN,2004-01-01,NaN,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://res.cloudinary.com/crunchbase-producti...
3,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,11.0,2007-05-26 04:22:15,2021-10-11 11:59:53,"investor,company",facebook.com,...,38.0,1.612282e+10,1.612282e+10,USD,2004-02-04,NaN,https://www.facebook.com/facebook,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://res.cloudinary.com/crunchbase-producti...
4,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,person,mark-zuckerberg,https://www.crunchbase.com/person/mark-zuckerberg,122070.0,2007-05-26 04:51:46,2021-10-23 05:57:44,investor,NaN,...,8.0,NaN,NaN,NaN,NaN,NaN,https://www.facebook.com/zuck,https://www.linkedin.com/in/mark-zuckerberg-61...,https://twitter.com/finkd,https://res.cloudinary.com/crunchbase-producti...


In [131]:
iv["idslice"] = iv["uuid"].apply(lambda x: x[-5:])
iv["name_id"] = iv["name"]  + "@" + iv["idslice"]
iv = iv[iv["name"].notna()]
iv["name_id"] = iv["name_id"].apply(lambda x: "_".join(x.split()))
iv.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,roles,domain,...,total_funding,total_funding_currency_code,founded_on,closed_on,facebook_url,linkedin_url,twitter_url,logo_url,idslice,name_id
0,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,person,ben-elowitz,https://www.crunchbase.com/person/ben-elowitz,221966.0,2007-05-25 14:03:54,2021-03-25 03:59:21,investor,wetpaint-inc.com,...,NaN,NaN,NaN,NaN,http://www.facebook.com/elowitz,https://www.linkedin.com/in/elowitz,http://twitter.com/elowitz,https://res.cloudinary.com/crunchbase-producti...,37a71,Ben_Elowitz@37a71
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,5453.0,2007-05-26 02:30:28,2018-10-27 00:29:49,"investor,company",zoho.com,...,NaN,NaN,1996-09-15,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://res.cloudinary.com/crunchbase-producti...,7efc7,Zoho@7efc7
2,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,146604.0,2007-05-26 03:21:34,2019-06-19 12:17:48,investor,omidyar.com,...,NaN,NaN,2004-01-01,NaN,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://res.cloudinary.com/crunchbase-producti...,a8eec,Omidyar_Network@a8eec
3,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,11.0,2007-05-26 04:22:15,2021-10-11 11:59:53,"investor,company",facebook.com,...,1.612282e+10,USD,2004-02-04,NaN,https://www.facebook.com/facebook,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://res.cloudinary.com/crunchbase-producti...,04fbb,Facebook@04fbb
4,a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,person,mark-zuckerberg,https://www.crunchbase.com/person/mark-zuckerberg,122070.0,2007-05-26 04:51:46,2021-10-23 05:57:44,investor,NaN,...,NaN,NaN,NaN,NaN,https://www.facebook.com/zuck,https://www.linkedin.com/in/mark-zuckerberg-61...,https://twitter.com/finkd,https://res.cloudinary.com/crunchbase-producti...,03f22,Mark_Zuckerberg@03f22


In [132]:
print(iv.columns)

Index(['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'roles', 'domain', 'country_code', 'state_code', 'region',
       'city', 'investor_types', 'investment_count', 'total_funding_usd',
       'total_funding', 'total_funding_currency_code', 'founded_on',
       'closed_on', 'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url',
       'idslice', 'name_id'],
      dtype='object')


In [133]:
if take_sample:
    iv = iv.loc[:int(sample_size*len(iv))]

In [134]:
iv_itype = pd.DataFrame(iv[["name_id", "investor_types"]])
#print(iv_itype.investor_types.unique()) # so there is "NaN"
iv_itype["investor_types"] = iv_itype["investor_types"].replace(float("NaN"), "investor")
iv_itype["investor_types"] = iv_itype["investor_types"].apply(lambda x: x.split(","))
iv_itype = iv_itype.explode("investor_types")
iv_itype.insert(1, "relation", "investor_type")
print(iv_itype.investor_types.unique())
iv_itype

['angel' 'investor' 'family_investment_office' 'investment_partner'
 'venture_capital' 'angel_group' 'private_equity_firm'
 'corporate_venture_capital' 'fund_of_funds' 'incubator' 'investment_bank'
 'micro_vc' 'government_office' 'hedge_fund' 'university_program'
 'entrepreneurship_program' 'venture_debt' 'accelerator'
 'co_working_space' 'pension_funds' 'secondary_purchaser'
 'startup_competition']


,name_id,relation,investor_types
0,Ben_Elowitz@37a71,investor_type,angel
1,Zoho@7efc7,investor_type,investor
2,Omidyar_Network@a8eec,investor_type,family_investment_office
3,Facebook@04fbb,investor_type,investor
4,Mark_Zuckerberg@03f22,investor_type,investment_partner
...,...,...,...
10019,Daniel_Keating_III@34d02,investor_type,angel
10020,Bryan_Mason@735d2,investor_type,angel
10021,Mark_Fruehan@cf11d,investor_type,investor
10022,Anurag_Chandra@2c3fc,investor_type,investor


In [135]:
iv_itype.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
iv_itype.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [136]:
#ivsubtype = list(iv_itype.investor_types.unique())
#ivsubtype.remove("investor")
#iv_subtype = pd.DataFrame(ivsubtype)
#iv_subtype.insert(1, "relation", "subtype_of")
#iv_subtype.insert(2, "type", "investor")
#print(iv_subtype)

In [137]:
#iv_subtype.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [138]:
iv_tf = pd.DataFrame(iv[["name_id", "total_funding_usd"]])
iv_tf = iv_tf.dropna()
iv_tf.insert(1, "relation", "total_invested_usd")
iv_tf

,name_id,relation,total_funding_usd
3,Facebook@04fbb,total_invested_usd,1.612282e+10
20,Twitter@1f709,total_invested_usd,1.540192e+09
37,eBay@2dcb1,total_invested_usd,6.700000e+06
57,Cisco@0065a,total_invested_usd,2.500000e+06
89,Prosper_Marketplace@7bbb6,total_invested_usd,4.154555e+08
...,...,...,...
9968,Bigpoint@79339,total_invested_usd,4.598708e+08
9980,Daimler@29485,total_invested_usd,1.369440e+09
9984,Union_Spring_Pharmaceuticals@74066,total_invested_usd,5.000000e+06
9988,Mint@4bbb7,total_invested_usd,3.177500e+07


In [139]:
iv_tf.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [140]:
column = 'total_funding_usd'
data = np.array(iv_tf[column].tolist()).reshape((len(iv_tf), 1))
select_k(data)

In [141]:
final_kmeans = KMeans(n_clusters=50).fit(data)
labels = ['cluster-{}-{}'.format(l, column) for l in final_kmeans.labels_]
mapping = dict(zip(data.squeeze().tolist(), labels))
iv_tf[column] = iv_tf[column].map(mapping)
iv_tf.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
dump_clusters(final_kmeans, data, column)
iv_tf

,name_id,relation,total_funding_usd
3,Facebook@04fbb,total_invested_usd,cluster-7-total_funding_usd
20,Twitter@1f709,total_invested_usd,cluster-4-total_funding_usd
37,eBay@2dcb1,total_invested_usd,cluster-0-total_funding_usd
57,Cisco@0065a,total_invested_usd,cluster-0-total_funding_usd
89,Prosper_Marketplace@7bbb6,total_invested_usd,cluster-6-total_funding_usd
...,...,...,...
9968,Bigpoint@79339,total_invested_usd,cluster-6-total_funding_usd
9980,Daimler@29485,total_invested_usd,cluster-4-total_funding_usd
9984,Union_Spring_Pharmaceuticals@74066,total_invested_usd,cluster-0-total_funding_usd
9988,Mint@4bbb7,total_invested_usd,cluster-0-total_funding_usd


In [142]:
iv_ic = pd.DataFrame(iv[["name_id", "investment_count"]])
iv_ic = iv_ic.dropna()
iv_ic.insert(1, "relation", "investment_count")
iv_ic

,name_id,relation,investment_count
0,Ben_Elowitz@37a71,investment_count,1.0
1,Zoho@7efc7,investment_count,6.0
2,Omidyar_Network@a8eec,investment_count,327.0
3,Facebook@04fbb,investment_count,38.0
4,Mark_Zuckerberg@03f22,investment_count,8.0
...,...,...,...
10017,La_Nacion@24600,investment_count,1.0
10018,Standard_Bank@29708,investment_count,14.0
10019,Daniel_Keating_III@34d02,investment_count,1.0
10020,Bryan_Mason@735d2,investment_count,1.0


In [143]:
iv_ic.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [145]:
column = 'investment_count'
data = np.array(iv_ic[column].tolist()).reshape((len(iv_ic), 1))
select_k(data)

In [146]:
final_kmeans = KMeans(n_clusters=60).fit(data)
labels = ['cluster-{}-{}'.format(l, column) for l in final_kmeans.labels_]
mapping = dict(zip(data.squeeze().tolist(), labels))
iv_ic[column] = iv_ic[column].map(mapping)
iv_ic.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
dump_clusters(final_kmeans, data, column)
iv_ic

,name_id,relation,investment_count
0,Ben_Elowitz@37a71,investment_count,cluster-0-investment_count
1,Zoho@7efc7,investment_count,cluster-0-investment_count
2,Omidyar_Network@a8eec,investment_count,cluster-7-investment_count
3,Facebook@04fbb,investment_count,cluster-8-investment_count
4,Mark_Zuckerberg@03f22,investment_count,cluster-0-investment_count
...,...,...,...
10017,La_Nacion@24600,investment_count,cluster-0-investment_count
10018,Standard_Bank@29708,investment_count,cluster-0-investment_count
10019,Daniel_Keating_III@34d02,investment_count,cluster-0-investment_count
10020,Bryan_Mason@735d2,investment_count,cluster-0-investment_count


#### Funds

In [147]:
f = pd.read_csv("data/funds.csv")
f.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,entity_uuid,entity_name,entity_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code
0,8190b28a-3c78-0ac4-1e7c-f0b5edce4b2e,DFJ Dragon Second Fund,fund,dfj-dragonfund-raised-dfj-dragon-second-fund--...,https://www.crunchbase.com/fund/dfj-dragonfund...,NaN,2008-12-17 03:07:16,2018-02-12 23:51:25,fe743289-a4aa-a470-00d8-bbe937e0955b,DFJ DragonFund,organization,2008-12-16,300000000.0,300000000.0,USD
1,cbb6644a-eafb-4686-5505-61f32e1c3638,Sequoia Israel Fund IV,fund,sequoia-capital-raised-sequoia-israel-fund-iv-...,https://www.crunchbase.com/fund/sequoia-capita...,NaN,2008-12-18 22:04:42,2018-02-13 00:38:07,0c867fde-2b9a-df10-fdb9-66b74f355f91,Sequoia Capital,organization,2008-12-17,200750000.0,200750000.0,USD
2,19fa5c4f-23c3-8c03-2b37-bcd0334505c9,InterWest Tenth Fund,fund,interwest-partners-raised-interwest-tenth-fund...,https://www.crunchbase.com/fund/interwest-part...,NaN,2008-12-31 09:47:51,2018-02-12 23:51:04,3eb9d040-1487-8d0e-4db8-72ec93ce32a5,InterWest Partners,organization,2008-08-11,650000000.0,650000000.0,USD
3,e803081d-0974-eda7-ab5f-c0f1a0a432fe,U.S. Venture Partners X,fund,us-venture-partners-raised-u-s--venture-partne...,https://www.crunchbase.com/fund/us-venture-par...,NaN,2009-01-01 18:13:44,2018-02-12 23:46:44,a2b35e8b-e3e9-883b-8a07-38e6ef57d726,U.S. Venture Partners,organization,2008-07-07,625000000.0,625000000.0,USD
4,49a5722a-084c-3466-fd90-37e885c800b1,Inventus Capital Partners Fund I LP,fund,inventus-capital-partners-raised-inventus-capi...,https://www.crunchbase.com/fund/inventus-capit...,NaN,2009-01-06 18:58:58,2018-02-12 23:35:11,c00d6191-2fbe-f796-54b3-fab074445635,Inventus Capital Partners,organization,2007-12-04,51750000.0,51750000.0,USD


In [148]:
f["id_slice"] = f["uuid"].apply(lambda x: x[-5:])
f["name_id"] =   f["name"]+ "@" + f["id_slice"]
f["name_id"] = f["name_id"].apply(lambda x: "_".join(x.split()))
f["eid_slice"] = f["entity_uuid"].apply(lambda x: x[-5:])
f["ename_id"] = f["entity_name"]+ "@" +f["eid_slice"]
f["ename_id"] = f["ename_id"].apply(lambda x: "_".join(x.split()))
f.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,entity_uuid,entity_name,entity_type,announced_on,raised_amount_usd,raised_amount,raised_amount_currency_code,id_slice,name_id,eid_slice,ename_id
0,8190b28a-3c78-0ac4-1e7c-f0b5edce4b2e,DFJ Dragon Second Fund,fund,dfj-dragonfund-raised-dfj-dragon-second-fund--...,https://www.crunchbase.com/fund/dfj-dragonfund...,NaN,2008-12-17 03:07:16,2018-02-12 23:51:25,fe743289-a4aa-a470-00d8-bbe937e0955b,DFJ DragonFund,organization,2008-12-16,300000000.0,300000000.0,USD,e4b2e,DFJ_Dragon_Second_Fund@e4b2e,0955b,DFJ_DragonFund@0955b
1,cbb6644a-eafb-4686-5505-61f32e1c3638,Sequoia Israel Fund IV,fund,sequoia-capital-raised-sequoia-israel-fund-iv-...,https://www.crunchbase.com/fund/sequoia-capita...,NaN,2008-12-18 22:04:42,2018-02-13 00:38:07,0c867fde-2b9a-df10-fdb9-66b74f355f91,Sequoia Capital,organization,2008-12-17,200750000.0,200750000.0,USD,c3638,Sequoia_Israel_Fund_IV@c3638,55f91,Sequoia_Capital@55f91
2,19fa5c4f-23c3-8c03-2b37-bcd0334505c9,InterWest Tenth Fund,fund,interwest-partners-raised-interwest-tenth-fund...,https://www.crunchbase.com/fund/interwest-part...,NaN,2008-12-31 09:47:51,2018-02-12 23:51:04,3eb9d040-1487-8d0e-4db8-72ec93ce32a5,InterWest Partners,organization,2008-08-11,650000000.0,650000000.0,USD,505c9,InterWest_Tenth_Fund@505c9,e32a5,InterWest_Partners@e32a5
3,e803081d-0974-eda7-ab5f-c0f1a0a432fe,U.S. Venture Partners X,fund,us-venture-partners-raised-u-s--venture-partne...,https://www.crunchbase.com/fund/us-venture-par...,NaN,2009-01-01 18:13:44,2018-02-12 23:46:44,a2b35e8b-e3e9-883b-8a07-38e6ef57d726,U.S. Venture Partners,organization,2008-07-07,625000000.0,625000000.0,USD,432fe,U.S._Venture_Partners_X@432fe,7d726,U.S._Venture_Partners@7d726
4,49a5722a-084c-3466-fd90-37e885c800b1,Inventus Capital Partners Fund I LP,fund,inventus-capital-partners-raised-inventus-capi...,https://www.crunchbase.com/fund/inventus-capit...,NaN,2009-01-06 18:58:58,2018-02-12 23:35:11,c00d6191-2fbe-f796-54b3-fab074445635,Inventus Capital Partners,organization,2007-12-04,51750000.0,51750000.0,USD,800b1,Inventus_Capital_Partners_Fund_I_LP@800b1,45635,Inventus_Capital_Partners@45635


In [149]:
#f_type = pd.DataFrame(f[["name_id"]])
#f_type.insert(1, "relation", "type_of")
#f_type.insert(2, "type", "fund")
#f_type

In [150]:
#f_type.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

In [151]:
if take_sample:
    f = f.loc[:int(sample_size*len(f))]

In [152]:
f_a = pd.DataFrame(f[["name_id", "announced_on"]])
f_a = f_a.dropna()
f_am = f_a.copy()
f_a["announced_on"] = f_a["announced_on"].apply(lambda x: x[:4])
f_a.insert(1, "relation", "announced_on_year")
f_a

,name_id,relation,announced_on
0,DFJ_Dragon_Second_Fund@e4b2e,announced_on_year,2008
1,Sequoia_Israel_Fund_IV@c3638,announced_on_year,2008
2,InterWest_Tenth_Fund@505c9,announced_on_year,2008
3,U.S._Venture_Partners_X@432fe,announced_on_year,2008
4,Inventus_Capital_Partners_Fund_I_LP@800b1,announced_on_year,2007
...,...,...,...
940,Parkwalk_UK_Technology_EIS_Fund_IV@e54a8,announced_on_year,2013
941,Parkwalk_Cambridge_Enterprise_Fund_I@ec934,announced_on_year,2012
942,Parkwalk_Cambridge_Enterprise_Fund_II@a10d2,announced_on_year,2013
943,Parkwalk_ZeroND_Fund@ceb01,announced_on_year,2013


In [153]:
f_a.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
f_a.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [154]:
f_am["announced_on"] = f_am["announced_on"].apply(lambda x: int(x[5:7]))
f_am.insert(1, "relation", "announced_on_month")
f_am

,name_id,relation,announced_on
0,DFJ_Dragon_Second_Fund@e4b2e,announced_on_month,12
1,Sequoia_Israel_Fund_IV@c3638,announced_on_month,12
2,InterWest_Tenth_Fund@505c9,announced_on_month,8
3,U.S._Venture_Partners_X@432fe,announced_on_month,7
4,Inventus_Capital_Partners_Fund_I_LP@800b1,announced_on_month,12
...,...,...,...
940,Parkwalk_UK_Technology_EIS_Fund_IV@e54a8,announced_on_month,4
941,Parkwalk_Cambridge_Enterprise_Fund_I@ec934,announced_on_month,5
942,Parkwalk_Cambridge_Enterprise_Fund_II@a10d2,announced_on_month,5
943,Parkwalk_ZeroND_Fund@ceb01,announced_on_month,3


In [155]:
f_am.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
f_am.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [156]:
f_e = f[[ "ename_id", "name_id"]]
f_e = f_e.dropna()
f_e.insert(1, "relation", "has_fund")
f_e

,ename_id,relation,name_id
0,DFJ_DragonFund@0955b,has_fund,DFJ_Dragon_Second_Fund@e4b2e
1,Sequoia_Capital@55f91,has_fund,Sequoia_Israel_Fund_IV@c3638
2,InterWest_Partners@e32a5,has_fund,InterWest_Tenth_Fund@505c9
3,U.S._Venture_Partners@7d726,has_fund,U.S._Venture_Partners_X@432fe
4,Inventus_Capital_Partners@45635,has_fund,Inventus_Capital_Partners_Fund_I_LP@800b1
...,...,...,...
940,Parkwalk_Advisors@d6a45,has_fund,Parkwalk_UK_Technology_EIS_Fund_IV@e54a8
941,Parkwalk_Advisors@d6a45,has_fund,Parkwalk_Cambridge_Enterprise_Fund_I@ec934
942,Parkwalk_Advisors@d6a45,has_fund,Parkwalk_Cambridge_Enterprise_Fund_II@a10d2
943,Parkwalk_Advisors@d6a45,has_fund,Parkwalk_ZeroND_Fund@ceb01


In [157]:
f_e.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
f_e.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [158]:
f_ra = f[["name_id", "raised_amount_usd"]]
f_ra = f_ra.dropna()
f_ra.insert(1, "relation", "raised_amount_usd")
f_ra

,name_id,relation,raised_amount_usd
0,DFJ_Dragon_Second_Fund@e4b2e,raised_amount_usd,300000000.0
1,Sequoia_Israel_Fund_IV@c3638,raised_amount_usd,200750000.0
2,InterWest_Tenth_Fund@505c9,raised_amount_usd,650000000.0
3,U.S._Venture_Partners_X@432fe,raised_amount_usd,625000000.0
4,Inventus_Capital_Partners_Fund_I_LP@800b1,raised_amount_usd,51750000.0
...,...,...,...
940,Parkwalk_UK_Technology_EIS_Fund_IV@e54a8,raised_amount_usd,4566856.0
941,Parkwalk_Cambridge_Enterprise_Fund_I@ec934,raised_amount_usd,1622629.0
942,Parkwalk_Cambridge_Enterprise_Fund_II@a10d2,raised_amount_usd,4667583.0
943,Parkwalk_ZeroND_Fund@ceb01,raised_amount_usd,1503508.0


In [159]:
f_ra.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [160]:
column = 'raised_amount_usd'
data = np.array(f_ra[column].tolist()).reshape((len(f_ra), 1))
select_k(data)

In [161]:
final_kmeans = KMeans(n_clusters=50).fit(data)
labels = ['cluster-{}-{}'.format(l, column) for l in final_kmeans.labels_]
mapping = dict(zip(data.squeeze().tolist(), labels))
f_ra[column] = f_ra[column].map(mapping)
f_ra.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
dump_clusters(final_kmeans, data, column)
f_ra

,name_id,relation,raised_amount_usd
0,DFJ_Dragon_Second_Fund@e4b2e,raised_amount_usd,cluster-3-raised_amount_usd
1,Sequoia_Israel_Fund_IV@c3638,raised_amount_usd,cluster-3-raised_amount_usd
2,InterWest_Tenth_Fund@505c9,raised_amount_usd,cluster-0-raised_amount_usd
3,U.S._Venture_Partners_X@432fe,raised_amount_usd,cluster-0-raised_amount_usd
4,Inventus_Capital_Partners_Fund_I_LP@800b1,raised_amount_usd,cluster-8-raised_amount_usd
...,...,...,...
940,Parkwalk_UK_Technology_EIS_Fund_IV@e54a8,raised_amount_usd,cluster-8-raised_amount_usd
941,Parkwalk_Cambridge_Enterprise_Fund_I@ec934,raised_amount_usd,cluster-8-raised_amount_usd
942,Parkwalk_Cambridge_Enterprise_Fund_II@a10d2,raised_amount_usd,cluster-8-raised_amount_usd
943,Parkwalk_ZeroND_Fund@ceb01,raised_amount_usd,cluster-8-raised_amount_usd


#### Acquisitions

In [162]:
a = pd.read_csv("data/acquisitions.csv")
a.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,acquiree_uuid,acquiree_name,...,acquirer_cb_url,acquirer_country_code,acquirer_state_code,acquirer_region,acquirer_city,acquisition_type,acquired_on,price_usd,price,price_currency_code
0,c62807f2-3487-f63b-e7fc-559b752ed44c,Flektor acquired by Fox Interactive Media,acquisition,fox-interactive-media-acquires-flektor--c62807f2,https://www.crunchbase.com/acquisition/fox-int...,120313.0,2007-05-31 22:19:54,2018-02-13 00:09:05,180ebf67-68d0-2316-e93d-8e1e546330ba,Flektor,...,https://www.crunchbase.com/organization/fox-in...,USA,CA,California,Beverly Hills,NaN,2007-05-30,NaN,NaN,NaN
1,419cabe1-4d71-5f64-3e9b-c9d0fb445cca,GrandCentral acquired by Google,acquisition,google-acquires-grandcentral--419cabe1,https://www.crunchbase.com/acquisition/google-...,30052.0,2007-07-03 08:14:50,2018-02-12 22:14:26,5b05e013-a448-3a0b-d872-a6ae668e1192,GrandCentral,...,https://www.crunchbase.com/organization/google,USA,CA,California,Mountain View,NaN,2007-07-01,60000000.0,60000000.0,USD
2,01bd2058-ccf8-6102-f253-9224a67a592b,Last.fm acquired by CBS Entertainment,acquisition,cbs-acquires-last-fm--01bd2058,https://www.crunchbase.com/acquisition/cbs-acq...,96102.0,2007-07-12 04:19:24,2018-02-12 23:41:20,8249dffa-1ca6-6f99-9f76-d56c83f85f2d,Last.fm,...,https://www.crunchbase.com/organization/cbs,USA,NY,New York,New York,NaN,2007-05-01,280000000.0,280000000.0,USD
3,831becd6-e913-b9f5-ac09-2bff915f9453,Kosiso - store acquired by Google,acquisition,google-acquires-feedburner--831becd6,https://www.crunchbase.com/acquisition/google-...,75010.0,2007-07-13 09:52:59,2018-02-12 23:43:38,10dd03fa-69ff-3a82-6321-c6b16c9a9f41,Kosiso - store,...,https://www.crunchbase.com/organization/google,USA,CA,California,Mountain View,acquisition,2007-05-23,100000000.0,100000000.0,USD
4,9da90ee7-aad7-e0b4-8456-839363032e5e,Recipezaar acquired by Scripps Networks,acquisition,scripps-networks-interactive-acquires-recipeza...,https://www.crunchbase.com/acquisition/scripps...,80392.0,2007-07-20 05:29:07,2018-02-12 23:51:07,0af10345-613d-e144-f8bd-b62e288985a0,Recipezaar,...,https://www.crunchbase.com/organization/scripp...,USA,TN,Tennessee,Knoxville,NaN,2007-07-01,NaN,NaN,NaN


In [163]:
a["uuid"] = a["uuid"].apply(lambda x: "ac@" + x)
a["ridslice"] = a["acquirer_uuid"].apply(lambda x: x[-5:])
a["rname_id"] = a["acquirer_name"] + "@" + a["ridslice"]
a["rname_id"]  = a["rname_id"] .apply(lambda x: "_".join(x.split()))
a["eidslice"] = a["acquiree_uuid"].apply(lambda x: x[-5:])
a["ename_id"] = a["acquiree_name"] + "@" + a["eidslice"]
a["ename_id"] = a["ename_id"].apply(lambda x: "_".join(x.split()))
a.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,acquiree_uuid,acquiree_name,...,acquirer_city,acquisition_type,acquired_on,price_usd,price,price_currency_code,ridslice,rname_id,eidslice,ename_id
0,ac@c62807f2-3487-f63b-e7fc-559b752ed44c,Flektor acquired by Fox Interactive Media,acquisition,fox-interactive-media-acquires-flektor--c62807f2,https://www.crunchbase.com/acquisition/fox-int...,120313.0,2007-05-31 22:19:54,2018-02-13 00:09:05,180ebf67-68d0-2316-e93d-8e1e546330ba,Flektor,...,Beverly Hills,NaN,2007-05-30,NaN,NaN,NaN,1b78b,Fox_Interactive_Media@1b78b,330ba,Flektor@330ba
1,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca,GrandCentral acquired by Google,acquisition,google-acquires-grandcentral--419cabe1,https://www.crunchbase.com/acquisition/google-...,30052.0,2007-07-03 08:14:50,2018-02-12 22:14:26,5b05e013-a448-3a0b-d872-a6ae668e1192,GrandCentral,...,Mountain View,NaN,2007-07-01,60000000.0,60000000.0,USD,27711,Google@27711,e1192,GrandCentral@e1192
2,ac@01bd2058-ccf8-6102-f253-9224a67a592b,Last.fm acquired by CBS Entertainment,acquisition,cbs-acquires-last-fm--01bd2058,https://www.crunchbase.com/acquisition/cbs-acq...,96102.0,2007-07-12 04:19:24,2018-02-12 23:41:20,8249dffa-1ca6-6f99-9f76-d56c83f85f2d,Last.fm,...,New York,NaN,2007-05-01,280000000.0,280000000.0,USD,2e4ea,CBS_Entertainment@2e4ea,85f2d,Last.fm@85f2d
3,ac@831becd6-e913-b9f5-ac09-2bff915f9453,Kosiso - store acquired by Google,acquisition,google-acquires-feedburner--831becd6,https://www.crunchbase.com/acquisition/google-...,75010.0,2007-07-13 09:52:59,2018-02-12 23:43:38,10dd03fa-69ff-3a82-6321-c6b16c9a9f41,Kosiso - store,...,Mountain View,acquisition,2007-05-23,100000000.0,100000000.0,USD,27711,Google@27711,a9f41,Kosiso_-_store@a9f41
4,ac@9da90ee7-aad7-e0b4-8456-839363032e5e,Recipezaar acquired by Scripps Networks,acquisition,scripps-networks-interactive-acquires-recipeza...,https://www.crunchbase.com/acquisition/scripps...,80392.0,2007-07-20 05:29:07,2018-02-12 23:51:07,0af10345-613d-e144-f8bd-b62e288985a0,Recipezaar,...,Knoxville,NaN,2007-07-01,NaN,NaN,NaN,a4b35,Scripps_Networks@a4b35,985a0,Recipezaar@985a0


In [164]:
if take_sample:
    a = a.loc[:int(sample_size*len(a))]

In [165]:
acquirer = pd.DataFrame(a[["rname_id", "uuid"]])
acquiree = pd.DataFrame(a[["ename_id", "uuid"]])
acquirer.insert(1, "relation", "is_acquirer")
acquiree.insert(1, "relation", "is_acquiree")
print(acquirer)
print(acquiree)

                               rname_id     relation  \
0           Fox_Interactive_Media@1b78b  is_acquirer   
1                          Google@27711  is_acquirer   
2               CBS_Entertainment@2e4ea  is_acquirer   
3                          Google@27711  is_acquirer   
4                Scripps_Networks@a4b35  is_acquirer   
...                                 ...          ...   
6355                CareerBuilder@1226a  is_acquirer   
6356            OnCourse_Learning@3ae1a  is_acquirer   
6357                     uromovie@d78d2  is_acquirer   
6358  Dominion_Business_Solutions@dacee  is_acquirer   
6359                         Deem@67469  is_acquirer   

                                         uuid  
0     ac@c62807f2-3487-f63b-e7fc-559b752ed44c  
1     ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca  
2     ac@01bd2058-ccf8-6102-f253-9224a67a592b  
3     ac@831becd6-e913-b9f5-ac09-2bff915f9453  
4     ac@9da90ee7-aad7-e0b4-8456-839363032e5e  
...                                    

In [166]:
acquiree.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
acquirer.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
acquiree.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)
acquirer.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [167]:
a_type1 = pd.DataFrame(a[["uuid", "acquisition_type"]])
a_type1 = a_type1.dropna()
a_type1.insert(1 ,"relation", "acquisition_type")
a_type1

,uuid,relation,acquisition_type
3,ac@831becd6-e913-b9f5-ac09-2bff915f9453,acquisition_type,acquisition
7,ac@b9ab8dbe-a12a-9ae5-b4e8-295d22d06c9d,acquisition_type,acquisition
8,ac@fbb1fd0a-f30e-78c8-035d-91d79f8c02cf,acquisition_type,acquisition
10,ac@10458e89-3e2d-74ba-5d4a-625b1a9b346c,acquisition_type,acquisition
12,ac@13bbd49b-5351-6b71-914e-8ba3cc0ecc7b,acquisition_type,acquisition
...,...,...,...
6344,ac@471bdd32-4022-1d96-ae34-cb0c697b02da,acquisition_type,acquisition
6346,ac@7f1726c4-08dd-59f1-9899-11d2cf0ff872,acquisition_type,acquisition
6349,ac@0a9a9dd9-3eb3-e477-a09e-28e2d1e4a250,acquisition_type,acquisition
6350,ac@ed9419db-2bc5-87de-0e4b-0a78e1a3e36d,acquisition_type,acquisition


In [168]:
a_type1.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
a_type1.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [169]:
a_pu = pd.DataFrame(a[["uuid", "price_usd"]])
a_pu = a_pu.dropna()
a_pu.insert(1 ,"relation", "price_usd")
a_pu

,uuid,relation,price_usd
1,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca,price_usd,6.000000e+07
2,ac@01bd2058-ccf8-6102-f253-9224a67a592b,price_usd,2.800000e+08
3,ac@831becd6-e913-b9f5-ac09-2bff915f9453,price_usd,1.000000e+08
8,ac@fbb1fd0a-f30e-78c8-035d-91d79f8c02cf,price_usd,1.500000e+09
10,ac@10458e89-3e2d-74ba-5d4a-625b1a9b346c,price_usd,5.000000e+06
...,...,...,...
6296,ac@b859598d-1f3c-4eb1-98da-a81376562636,price_usd,1.440000e+08
6299,ac@5f169aef-fe2d-c83d-38a9-78dd2f15b381,price_usd,1.350000e+08
6304,ac@2273a9cf-4718-3f93-4d90-cccf3714934a,price_usd,3.350000e+08
6313,ac@d57d54bc-5c34-a1ed-27c5-52795b785548,price_usd,3.700000e+09


In [170]:
a_pu.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [172]:
column = 'price_usd'
data = np.array(a_pu[column].tolist()).reshape((len(a_pu), 1))
select_k(data)

In [173]:
final_kmeans = KMeans(n_clusters=60).fit(data)
labels = ['cluster-{}-{}'.format(l, column) for l in final_kmeans.labels_]
mapping = dict(zip(data.squeeze().tolist(), labels))
a_pu[column] = a_pu[column].map(mapping)
a_pu.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
dump_clusters(final_kmeans, data, column)
a_pu

,uuid,relation,price_usd
1,ac@419cabe1-4d71-5f64-3e9b-c9d0fb445cca,price_usd,cluster-0-price_usd
2,ac@01bd2058-ccf8-6102-f253-9224a67a592b,price_usd,cluster-7-price_usd
3,ac@831becd6-e913-b9f5-ac09-2bff915f9453,price_usd,cluster-0-price_usd
8,ac@fbb1fd0a-f30e-78c8-035d-91d79f8c02cf,price_usd,cluster-5-price_usd
10,ac@10458e89-3e2d-74ba-5d4a-625b1a9b346c,price_usd,cluster-0-price_usd
...,...,...,...
6296,ac@b859598d-1f3c-4eb1-98da-a81376562636,price_usd,cluster-0-price_usd
6299,ac@5f169aef-fe2d-c83d-38a9-78dd2f15b381,price_usd,cluster-0-price_usd
6304,ac@2273a9cf-4718-3f93-4d90-cccf3714934a,price_usd,cluster-7-price_usd
6313,ac@d57d54bc-5c34-a1ed-27c5-52795b785548,price_usd,cluster-3-price_usd


In [ ]:
#a_subtype = pd.DataFrame(a_type1.acquisition_type.unique())
#a_subtype.columns = ["acquisition_type"]
#a_subtype.insert(1, "relation", "subtype_of")
#a_subtype.insert(2, "type","acquisition")
#print(a_subtype)

In [ ]:
#a_subtype.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)

### Events

In [174]:
e = pd.read_csv("data/events.csv")
e.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,short_description,started_on,...,event_url,registration_url,venue_name,description,country_code,state_code,region,city,logo_url,event_roles
0,6a21f93a-3cae-02ac-44c7-c166c0ed50fa,SF New Tech,event,sfnt-201272-san-francisco,https://www.crunchbase.com/event/sfnt-201272-s...,17183.0,2014-10-03 18:12:25,2018-02-13 01:14:28,"Join us on Wed, Oct 15th for another edition o...",2014-10-14,...,NaN,http://1015sfnt-meetup.eventbrite.com,RocketStudios,NaN,USA,CA,California,San Francisco,NaN,conference
1,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,event,adobe-max-2014104-los-angeles,https://www.crunchbase.com/event/adobe-max-201...,14386.0,2014-10-03 18:14:15,2018-02-13 01:14:24,"Sessions, labs, and creativity workshops where...",2014-10-04,...,NaN,http://max.adobe.com/registration/,Los Angeles Convention Center,NaN,USA,CA,California,Los Angeles,https://res.cloudinary.com/crunchbase-producti...,conference
2,c6006d9f-e796-2a2a-2efc-c4e55caa98f8,ad:tech New York,event,ad-tech-new-york-2014105-new-york,https://www.crunchbase.com/event/ad-tech-new-y...,3398.0,2014-10-03 18:40:03,2018-02-13 01:14:23,ad:tech New York will showcase keynote speaker...,2014-10-05,...,NaN,http://ad-tech.com/ny/registration,Javits Center,NaN,USA,NY,New York,New York,https://res.cloudinary.com/crunchbase-producti...,conference
3,2710319a-f022-5cd3-f855-6594fd0d6db3,Engadget Expand,event,engadget-expand-2014107-new-york,https://www.crunchbase.com/event/engadget-expa...,15995.0,2014-10-03 18:51:28,2018-02-13 01:14:21,Engadget Expand is designed from the ground up...,2014-11-07,...,NaN,http://www.eventbrite.com/e/engadget-expand-ny...,Javits Center,"It’s a free, 10,000+ attendee event that give...",USA,NY,New York,New York,https://res.cloudinary.com/crunchbase-producti...,"conference,expo"
4,35e61ac8-58bf-0492-8285-39dc9c2b236d,Dreamforce 2014,event,dreamforce-20141013-san-francisco,https://www.crunchbase.com/event/dreamforce-20...,2350.0,2014-10-03 19:05:07,2018-06-22 04:56:39,Dreamforce is your chance to learn from indust...,2014-10-13,...,https://www.montypythonlive.com/best-led-grow-...,http://www.salesforce.com/dreamforce/DF14/regi...,Moscone Center,NaN,USA,CA,California,San Francisco,https://res.cloudinary.com/crunchbase-producti...,conference


In [175]:
e = e.drop_duplicates(subset = "name", keep = "last")
e["name"] = e["name"].apply(lambda x: "_".join(x.split()))
e.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,short_description,started_on,...,event_url,registration_url,venue_name,description,country_code,state_code,region,city,logo_url,event_roles
0,6a21f93a-3cae-02ac-44c7-c166c0ed50fa,SF_New_Tech,event,sfnt-201272-san-francisco,https://www.crunchbase.com/event/sfnt-201272-s...,17183.0,2014-10-03 18:12:25,2018-02-13 01:14:28,"Join us on Wed, Oct 15th for another edition o...",2014-10-14,...,NaN,http://1015sfnt-meetup.eventbrite.com,RocketStudios,NaN,USA,CA,California,San Francisco,NaN,conference
1,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe_MAX_2014,event,adobe-max-2014104-los-angeles,https://www.crunchbase.com/event/adobe-max-201...,14386.0,2014-10-03 18:14:15,2018-02-13 01:14:24,"Sessions, labs, and creativity workshops where...",2014-10-04,...,NaN,http://max.adobe.com/registration/,Los Angeles Convention Center,NaN,USA,CA,California,Los Angeles,https://res.cloudinary.com/crunchbase-producti...,conference
2,c6006d9f-e796-2a2a-2efc-c4e55caa98f8,ad:tech_New_York,event,ad-tech-new-york-2014105-new-york,https://www.crunchbase.com/event/ad-tech-new-y...,3398.0,2014-10-03 18:40:03,2018-02-13 01:14:23,ad:tech New York will showcase keynote speaker...,2014-10-05,...,NaN,http://ad-tech.com/ny/registration,Javits Center,NaN,USA,NY,New York,New York,https://res.cloudinary.com/crunchbase-producti...,conference
3,2710319a-f022-5cd3-f855-6594fd0d6db3,Engadget_Expand,event,engadget-expand-2014107-new-york,https://www.crunchbase.com/event/engadget-expa...,15995.0,2014-10-03 18:51:28,2018-02-13 01:14:21,Engadget Expand is designed from the ground up...,2014-11-07,...,NaN,http://www.eventbrite.com/e/engadget-expand-ny...,Javits Center,"It’s a free, 10,000+ attendee event that give...",USA,NY,New York,New York,https://res.cloudinary.com/crunchbase-producti...,"conference,expo"
5,40d5154d-4e8f-342e-b83e-93b191336c3e,Wired_2014,event,wired-2014-20141016-london,https://www.crunchbase.com/event/wired-2014-20...,23215.0,2014-10-03 19:10:15,2018-02-13 01:14:23,WIRED2014 aims to bring the WIRED world to lif...,2014-10-16,...,NaN,https://www.eventbrite.co.uk/e/wired-2014-tick...,Tobacco Dock,NaN,GBR,NaN,England,London,https://res.cloudinary.com/crunchbase-producti...,conference


In [176]:
if take_sample:
    e = e.loc[:int(sample_size*len(e))]

In [177]:
e_start = pd.DataFrame(e[["name", "started_on"]])
e_start = e_start.dropna()
e_start_month = e_start.copy()
e_start["started_on"] = e_start["started_on"].apply(lambda x: x[:4])
e_start.insert(1, "relation", "started_on_year")
e_start

,name,relation,started_on
0,SF_New_Tech,started_on_year,2014
1,Adobe_MAX_2014,started_on_year,2014
2,ad:tech_New_York,started_on_year,2014
3,Engadget_Expand,started_on_year,2014
5,Wired_2014,started_on_year,2014
...,...,...,...
1156,Virtual_Piggy_Hackathon,started_on_year,2013
1160,Orrick_Total_Access_MA_and_IPO_Market_Update,started_on_year,2014
1161,NWA_TechFest,started_on_year,2014
1162,500_Startups_Commercism_Conference,started_on_year,2014


In [178]:
e_start.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
e_start.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [179]:
e_start_month["started_on"] = e_start_month["started_on"].apply(lambda x: int(x[5:7]))
e_start_month.insert(1, "relation", "started_on_month")
e_start_month

,name,relation,started_on
0,SF_New_Tech,started_on_month,10
1,Adobe_MAX_2014,started_on_month,10
2,ad:tech_New_York,started_on_month,10
3,Engadget_Expand,started_on_month,11
5,Wired_2014,started_on_month,10
...,...,...,...
1156,Virtual_Piggy_Hackathon,started_on_month,12
1160,Orrick_Total_Access_MA_and_IPO_Market_Update,started_on_month,1
1161,NWA_TechFest,started_on_month,1
1162,500_Startups_Commercism_Conference,started_on_month,3


In [180]:
e_start_month.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
e_start_month.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [181]:
e_end = pd.DataFrame(e[["name", "ended_on"]])
e_end = e_end.dropna()
e_end_month = e_end.copy()
e_end["ended_on"] = e_end["ended_on"].apply(lambda x: x[:4])
e_end.insert(1, "relation", "ended_on_year")
e_end

,name,relation,ended_on
0,SF_New_Tech,ended_on_year,2014
1,Adobe_MAX_2014,ended_on_year,2014
2,ad:tech_New_York,ended_on_year,2014
3,Engadget_Expand,ended_on_year,2014
5,Wired_2014,ended_on_year,2014
...,...,...,...
1156,Virtual_Piggy_Hackathon,ended_on_year,2013
1160,Orrick_Total_Access_MA_and_IPO_Market_Update,ended_on_year,2014
1161,NWA_TechFest,ended_on_year,2014
1162,500_Startups_Commercism_Conference,ended_on_year,2014


In [182]:
e_end.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
e_end.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [183]:
e_end_month["ended_on"] = e_end_month["ended_on"].apply(lambda x: int(x[5:7]))
e_end_month.insert(1, "relation", "ended_on_month")
e_end_month

,name,relation,ended_on
0,SF_New_Tech,ended_on_month,10
1,Adobe_MAX_2014,ended_on_month,10
2,ad:tech_New_York,ended_on_month,10
3,Engadget_Expand,ended_on_month,11
5,Wired_2014,ended_on_month,10
...,...,...,...
1156,Virtual_Piggy_Hackathon,ended_on_month,12
1160,Orrick_Total_Access_MA_and_IPO_Market_Update,ended_on_month,1
1161,NWA_TechFest,ended_on_month,1
1162,500_Startups_Commercism_Conference,ended_on_month,3


In [184]:
e_end_month.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
e_end_month.to_csv("literals.txt", sep='\t', mode='a', index=False, header = None)

In [185]:
e_c = pd.DataFrame(e[["name", "country_code"]])
e_c = e_c.dropna()
e_c.insert(1, "relation", "country_code")
e_c

,name,relation,country_code
0,SF_New_Tech,country_code,USA
1,Adobe_MAX_2014,country_code,USA
2,ad:tech_New_York,country_code,USA
3,Engadget_Expand,country_code,USA
5,Wired_2014,country_code,GBR
...,...,...,...
1156,Virtual_Piggy_Hackathon,country_code,USA
1160,Orrick_Total_Access_MA_and_IPO_Market_Update,country_code,USA
1161,NWA_TechFest,country_code,USA
1162,500_Startups_Commercism_Conference,country_code,USA


In [186]:
e_c.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
e_c.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [187]:
e_city = pd.DataFrame(e[["name", "city"]])
e_city = e_city.dropna()
e_city["city"] = e_city["city"].apply(lambda x: "_".join(x.split()))
e_city.insert(1, "relation", "city")
e_city

,name,relation,city
0,SF_New_Tech,city,San_Francisco
1,Adobe_MAX_2014,city,Los_Angeles
2,ad:tech_New_York,city,New_York
3,Engadget_Expand,city,New_York
5,Wired_2014,city,London
...,...,...,...
1156,Virtual_Piggy_Hackathon,city,Los_Angeles
1160,Orrick_Total_Access_MA_and_IPO_Market_Update,city,Menlo_Park
1161,NWA_TechFest,city,Fayetteville
1162,500_Startups_Commercism_Conference,city,Mountain_View


In [188]:
e_city.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
e_city.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [189]:
e_role = pd.DataFrame(e[["name", "event_roles"]])
e_role["event_roles"] = e_role["event_roles"].replace("other", float('Nan'))
e_role = e_role.dropna()
e_role["event_roles"] = e_role["event_roles"].apply(lambda x: x.split(","))
e_role = e_role.explode("event_roles")
e_role["event_roles"] = e_role["event_roles"].apply(lambda x: x.strip())
e_role.insert(1, "relation", "event_type")
e_role

,name,relation,event_roles
0,SF_New_Tech,event_type,conference
1,Adobe_MAX_2014,event_type,conference
2,ad:tech_New_York,event_type,conference
3,Engadget_Expand,event_type,conference
3,Engadget_Expand,event_type,expo
...,...,...,...
1156,Virtual_Piggy_Hackathon,event_type,hackathon
1160,Orrick_Total_Access_MA_and_IPO_Market_Update,event_type,conference
1161,NWA_TechFest,event_type,conference
1162,500_Startups_Commercism_Conference,event_type,conference


In [190]:
e_role.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
e_role.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [191]:
#eroles = list(e_role["event_roles"].unique())
#eroles.remove("event")
#print(eroles)
#e_sub = pd.DataFrame(eroles)
#e_sub.insert(1, "relation", "subtype_of")
#e_sub.insert(2, "type", "event")
#e_sub

In [192]:
#e_sub.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
#e_sub.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

#### Event appearance

In [193]:
ea = pd.read_csv("data/event_appearances.csv")
ea.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,event_uuid,event_name,participant_uuid,participant_name,participant_type,appearance_type,short_description
0,30c4023d-b694-4704-97d7-cd99cd06ecce,Adobe MAX 2014's sponsor - CDW Corporation,event_appearance,cdw-corporation-sponsor-adobe-max-2014104-los-...,https://www.crunchbase.com/event_appearance/cd...,NaN,2014-10-03 18:21:26,2018-02-13 01:16:54,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,ae883a31-3739-7eb9-4ce0-463ee070ed79,CDW Corporation,organization,sponsor,NaN
1,dffe0542-3dac-4228-a997-3eed42cbd67e,Adobe MAX 2014's sponsor - Microsoft,event_appearance,microsoft-sponsor-adobe-max-2014104-los-angele...,https://www.crunchbase.com/event_appearance/mi...,NaN,2014-10-03 18:21:42,2018-02-13 01:16:21,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,fd80725f-53fc-7009-9878-aeecf1e9ffbb,Microsoft,organization,sponsor,NaN
2,21884e6c-d097-4c54-8513-10536a0ea60d,Adobe MAX 2014's sponsor - Twitter,event_appearance,twitter-sponsor-adobe-max-2014104-los-angeles-...,https://www.crunchbase.com/event_appearance/tw...,NaN,2014-10-03 18:21:55,2018-02-13 01:16:23,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,Twitter,organization,sponsor,NaN
3,c3782f8f-4684-47d6-8649-d0b3e1825eb5,Adobe MAX 2014's sponsor - HP,event_appearance,hewlett-packard-sponsor-adobe-max-2014104-los-...,https://www.crunchbase.com/event_appearance/he...,NaN,2014-10-03 18:22:03,2018-02-13 01:16:54,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,8adadbfb-be63-6602-8ca0-b037397a7038,HP,organization,sponsor,NaN
4,55e2ee8b-b5b7-49d3-bd21-a63eedf6b7f2,Adobe MAX 2014's sponsor - Intel,event_appearance,intel-sponsor-adobe-max-2014104-los-angeles--5...,https://www.crunchbase.com/event_appearance/in...,NaN,2014-10-03 18:22:11,2018-02-13 01:16:24,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe MAX 2014,1e4f199c-363b-451b-a164-f94571075ee5,Intel,organization,sponsor,NaN


In [194]:
ea = ea.drop_duplicates(subset = "name", keep = "last")
ea["idslice"] = ea["participant_uuid"].apply(lambda x: x[-5:])
ea["name_id"] = ea["participant_name"] + "@" + ea["idslice"] 
ea["name_id"] = ea["name_id"].apply(lambda x: "_".join(x.split()))
ea["event_name"] = ea["event_name"].apply(lambda x: "_".join(x.split()))
ea.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,event_uuid,event_name,participant_uuid,participant_name,participant_type,appearance_type,short_description,idslice,name_id
0,30c4023d-b694-4704-97d7-cd99cd06ecce,Adobe MAX 2014's sponsor - CDW Corporation,event_appearance,cdw-corporation-sponsor-adobe-max-2014104-los-...,https://www.crunchbase.com/event_appearance/cd...,NaN,2014-10-03 18:21:26,2018-02-13 01:16:54,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe_MAX_2014,ae883a31-3739-7eb9-4ce0-463ee070ed79,CDW Corporation,organization,sponsor,NaN,0ed79,CDW_Corporation@0ed79
1,dffe0542-3dac-4228-a997-3eed42cbd67e,Adobe MAX 2014's sponsor - Microsoft,event_appearance,microsoft-sponsor-adobe-max-2014104-los-angele...,https://www.crunchbase.com/event_appearance/mi...,NaN,2014-10-03 18:21:42,2018-02-13 01:16:21,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe_MAX_2014,fd80725f-53fc-7009-9878-aeecf1e9ffbb,Microsoft,organization,sponsor,NaN,9ffbb,Microsoft@9ffbb
2,21884e6c-d097-4c54-8513-10536a0ea60d,Adobe MAX 2014's sponsor - Twitter,event_appearance,twitter-sponsor-adobe-max-2014104-los-angeles-...,https://www.crunchbase.com/event_appearance/tw...,NaN,2014-10-03 18:21:55,2018-02-13 01:16:23,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe_MAX_2014,5da6106f-0d27-0d37-e9d7-dcfeccc1f709,Twitter,organization,sponsor,NaN,1f709,Twitter@1f709
3,c3782f8f-4684-47d6-8649-d0b3e1825eb5,Adobe MAX 2014's sponsor - HP,event_appearance,hewlett-packard-sponsor-adobe-max-2014104-los-...,https://www.crunchbase.com/event_appearance/he...,NaN,2014-10-03 18:22:03,2018-02-13 01:16:54,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe_MAX_2014,8adadbfb-be63-6602-8ca0-b037397a7038,HP,organization,sponsor,NaN,a7038,HP@a7038
4,55e2ee8b-b5b7-49d3-bd21-a63eedf6b7f2,Adobe MAX 2014's sponsor - Intel,event_appearance,intel-sponsor-adobe-max-2014104-los-angeles--5...,https://www.crunchbase.com/event_appearance/in...,NaN,2014-10-03 18:22:11,2018-02-13 01:16:24,135a927a-b238-037e-50e2-d3a2e3511ed2,Adobe_MAX_2014,1e4f199c-363b-451b-a164-f94571075ee5,Intel,organization,sponsor,NaN,75ee5,Intel@75ee5


In [195]:
if take_sample:
    ea = ea.loc[:int(sample_size*len(ea))]

In [196]:
ea_atype = pd.DataFrame(ea[["name_id", "appearance_type", "event_name"]])
ea_atype = ea_atype.dropna()
ea_atype

,name_id,appearance_type,event_name
0,CDW_Corporation@0ed79,sponsor,Adobe_MAX_2014
1,Microsoft@9ffbb,sponsor,Adobe_MAX_2014
2,Twitter@1f709,sponsor,Adobe_MAX_2014
3,HP@a7038,sponsor,Adobe_MAX_2014
4,Intel@75ee5,sponsor,Adobe_MAX_2014
...,...,...,...
20940,Nandan_Nilekani@8a97e,speaker,Skoll_World_Forum
20941,Marc_P._Bernegger@d04d5,speaker,Start_Summit_15
20942,Caty_Borum_Chattoo@bcf35,speaker,Skoll_World_Forum
20943,Alexander_Müller@f8cf4,speaker,Start_Summit_15


In [197]:
ea_atype.to_csv("triplets.txt", sep='\t', mode='a', index=False, header = None)
ea_atype.to_csv("triplets_without_literals.txt", sep='\t', mode='a', index=False, header = None)

In [198]:
with open('clusters.pickle', 'wb') as f:
    pickle.dump(clusters, f)